In [ ]:
#05 统计bayes的每个样本reads数，写成A256G的形式，过滤，只过滤测序深度,不过滤频率。
#统计每个样本的突变数量
#dp>100并且在至少2个样本中出现过的才算。
#变异类型是snp或者del或ins，这里也筛选了。
#最终输出csv文件。

import csv
import os
import vcf
wenjianjia_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/yjq_sra_bioproject_rsv')
for i in wenjianjia_list:
    dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/01filtered_csv_deep_sample/{i}'
    if not os.path.exists(dirs):
        os.makedirs(dirs)

for p in os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/vcf_pair'):
    prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/vcf_pair/{p}'
    
    bayes_filenames=os.listdir(prj_path)
    type_list= []
    dict_mut_num = {}#在几个样本中出现过。形成列表，在列表中的才算真正的突变。
    #找出突变在几个样本中出现
    for i in bayes_filenames:
        if i.endswith('vcf'):
            i_1 = i.split('.')[0]
            reads_pos_dict = {}
            path1 = prj_path+'/'+i
            vcf_reader = vcf.Reader(filename=path1)
            for record in vcf_reader:
    #             print(record)
    #             print(record.INFO)

                alt_count = record.INFO['AO']
                ref_count = record.INFO['RO']
                ref = str(record).split(',')[2].split('=')[1]
                type0 = record.INFO['TYPE']
                position = str(record).split(',')[1].split('=')[1]
                dp = record.INFO['DP']
                if type0 not in type_list:
                    type_list.append(type0)
                for i in range(0,len(record.INFO['TYPE'])):
                    alt1 = str(record).split('=')[4].replace(')','').replace('[','').replace(']','').replace(' ','')
                    alt2 = alt1.split(",")#转成列表,碱基
                    type2 = record.INFO['TYPE'][i]
                    info = f'{ref}_{position}_{alt2[i]}'
                    alt_count1 = alt_count[i]
                    af = record.INFO['AO'][i]/(record.INFO['AO'][i] + record.INFO['RO'])
    #                 print(af)
                    if dp > 100:
    #                     reads_pos_dict[info] = {'alt':alt_count1,'ref':ref_count,'type':type2,'allele_frequency':af}
                        if info not in dict_mut_num:
                            dict_mut_num[info] = 1
                        else:
                            dict_mut_num[info] += 1
    #         print(reads_pos_dict)



    filtered_list = []# 测序深度>100且出现在至少2个样本中的突变。
    for k,v in dict_mut_num.items():
        if v > 1:
            filtered_list.append(k)
    # print(filtered_list)


    #根据filtered_list筛选突变，输出csv。
    for i in bayes_filenames:
        if i.endswith('vcf'):
            i_1 = i.split('.')[0]
            reads_pos_dict = {}
            path1 = prj_path +'/'+i
            vcf_reader = vcf.Reader(filename=path1)
            for record in vcf_reader:
    #             print(record)
    #             print(record.INFO)

                alt_count = record.INFO['AO']
                ref_count = record.INFO['RO']
                ref = str(record).split(',')[2].split('=')[1]
                type0 = record.INFO['TYPE']
                position = str(record).split(',')[1].split('=')[1]
                dp = record.INFO['DP']
                if type0 not in type_list:
                    type_list.append(type0)
                for i in range(0,len(record.INFO['TYPE'])):
                    alt1 = str(record).split('=')[4].replace(')','').replace('[','').replace(']','').replace(' ','')
                    alt2 = alt1.split(",")#转成列表,碱基
                    type2 = record.INFO['TYPE'][i]
                    info = f'{ref}_{position}_{alt2[i]}'
                    alt_count1 = alt_count[i]
                    af = record.INFO['AO'][i]/(record.INFO['AO'][i] + record.INFO['RO'])    
                    if dp > 100:#即使有列表了也还是需要测序深度>100的这个条件，不然有可能是出现在样本多但是测序深度不行的。
                        if info in filtered_list:
                            if type2 == 'snp' or type2 == 'del' or type2 == 'ins':#筛选变异类型
                                reads_pos_dict[info] = {'alt':alt_count1,'ref':ref_count,'type':type2,'allele_frequency':af,'sample_exist':dict_mut_num[info]}
    #         print(reads_pos_dict)

            with open(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/01filtered_csv_deep_sample/{p}/{i_1}.csv','w') as f:#表格可以加个频率
                writer = csv.writer(f)
                writer.writerow(["mutation","alt_reads","ref_reads","type","app","allele_frequency","sample_exist'"])
                for k,v in reads_pos_dict.items():
                    writer.writerow([k,v['alt'],v['ref'],v['type'],'freebayes',v['allele_frequency'],v['sample_exist']])

In [ ]:
!pip install pyvcf

In [ ]:
#04统计突变类型，3%-80%和80%以上的比例，先不管allele frequency，只看次数
#二代测序中3%-80%的突变，对频率有错误影响，对蛋白分布没有影响。可以加一列属于哪个样本的。加上了，次数是准的了。

#80%以下：
import os
import csv

# wenjianjia_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/yjq_sra_bioproject_rsv')
# for i in wenjianjia_list:
#     dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/06product_exist/{i}'
#     if not os.path.exists(dirs):
#         os.makedirs(dirs)

# wenjianjia_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/yjq_sra_bioproject_rsv')
# for i in wenjianjia_list:
#     dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/07mutation_proportion/{i}'
#     if not os.path.exists(dirs):
#         os.makedirs(dirs)

for p in os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/01filtered_csv_deep_sample'):
    prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/01filtered_csv_deep_sample/{p}'

    dict_003_08_next_generation = {}
    for i in os.listdir(prj_path):
        if i.endswith('.csv'):
            csv_path = f'{prj_path}/{i}'
            with open(csv_path) as csvfile:
                csv_reader = csv.reader(csvfile)  
                for row in csv_reader:
                    if row[0] != 'mutation':
                        first = row[0].split('_')[0]
    #                     last = row[0].split('_')[2]
    #                     position = row[0].split('_')[1]
                        frequnecy = float(row[5])
                        mutation = row[0]
                        type1 = row[3]
                        if 0.03 <= frequnecy <= 0.8:
    #                         if len(first) == 1:#找的snv很奇怪，明明是单个突变非要写一堆，所以就不用type筛选了。
    #                             dict_003_08_next_generation[mutation] = frequnecy
                            if mutation not in dict_003_08_next_generation.keys():
                                dict_003_08_next_generation[mutation] = {}
                                dict_003_08_next_generation[mutation][i] = [frequnecy,type1]
                            else:#一定要加，不然每次都是赋值一个空字典就没办法新加进来样本。
                                dict_003_08_next_generation[mutation][i] = [frequnecy,type1]
    # print(dict_003_08_next_generation)
    # print(p)
# ----------------------------------------------------------------------------------------------------------    
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!统计所有的突变及其数量，频率。 后面看在哪个突变上要用。             
    with open(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/06product_exist/06isnv_mutation_{p}.csv','w') as f:
        writer = csv.writer(f)
        writer.writerow(['isnv','sample','frequency','type'])
        for k,v in dict_003_08_next_generation.items():
            for s,info in v.items():
                frequency = info[0]
                type3 = info[1]
                writer.writerow([k,s.split('.')[0],frequency,type3])
            
#-----------------------------------------------------------------------------------------------------------
    
#统计数量
    dict_num_mutation = {'A_G':0,'A_T':0,'A_C':0,'G_A':0,'G_C':0,'G_T':0,'T_C':0,'T_A':0,'T_G':0,'C_A':0,'C_T':0,'C_G':0,'del':0,'ins':0}
    for k,v in dict_003_08_next_generation.items():
        ref = k.split('_')[0]
        alt = k.split('_')[2]
        for s in v.values():
            type2 = s[1]
    #         print(type2)
        if type2 =='snp':
            name = f'{ref}_{alt}'
            add_count = len(v)
        #     print(name)
            for m,c in dict_num_mutation.items():
                if name == m:
                    dict_num_mutation[m] += add_count  #!!!赋值不可以只写c，一定要写成键的值。
        if type2 =='del':
            add_count = len(v)

            dict_num_mutation['del'] += add_count
        if type2 =='ins':
            add_count = len(v)

            dict_num_mutation['ins'] += add_count



    print(dict_num_mutation)
    count_all = 0
    for k,v in dict_num_mutation.items():
        count_all += v
    print(count_all)
    
    dict_proportion =  dict.fromkeys(dict_num_mutation.keys())
    for k in dict_proportion.keys():
        dict_proportion[k] = {}
        dict_proportion[k]['count'] = dict_num_mutation[k]
        if count_all!= 0:
            dict_proportion[k]['proportion'] = dict_proportion[k]['count']/count_all
        else:
            dict_proportion[k]['proportion'] = '/'
    print(dict_proportion)  
    with open(f"/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/07mutation_proportion/07isnv_mutation_type_{p}.csv","w") as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerow(['mutation_type','count','proportion','group'])
        for k,v in dict_proportion.items():
            writer.writerow([k,v['count'],v['proportion'],'iSNV'])

        
        
        
        
        
# #-----------------------------------------------------------------------
# #80%上：

    dict_003_08_next_generation = {}
    for i in os.listdir(prj_path):
        if i.endswith('.csv'):
            csv_path = f'{prj_path}/{i}'
            with open(csv_path) as csvfile:
                csv_reader = csv.reader(csvfile)  
                for row in csv_reader:
                    if row[0] != 'mutation':
                        first = row[0].split('_')[0]
    #                     last = row[0].split('_')[2]
    #                     position = row[0].split('_')[1]
                        frequnecy = float(row[5])
                        mutation = row[0]
                        type1 = row[3]
                        if  frequnecy > 0.8:
    #                         if len(first) == 1:#找的snv很奇怪，明明是单个突变非要写一堆，所以就不用type筛选了。
    #                             dict_003_08_next_generation[mutation] = frequnecy
                            if mutation not in dict_003_08_next_generation.keys():
                                dict_003_08_next_generation[mutation] = {}
                                dict_003_08_next_generation[mutation][i] = [frequnecy,type1]
                            else:#一定要加，不然每次都是赋值一个空字典就没办法新加进来样本。
                                dict_003_08_next_generation[mutation][i] = [frequnecy,type1]

    #统计数量
    dict_num_mutation = {'A_G':0,'A_T':0,'A_C':0,'G_A':0,'G_C':0,'G_T':0,'T_C':0,'T_A':0,'T_G':0,'C_A':0,'C_T':0,'C_G':0,'del':0,'ins':0}
    for k,v in dict_003_08_next_generation.items():
        ref = k.split('_')[0]
        alt = k.split('_')[2]
        for s in v.values():
            type2 = s[1]
    #         print(type2)
        if type2 =='snp':
            name = f'{ref}_{alt}'
            add_count = len(v)
        #     print(name)
            for m,c in dict_num_mutation.items():
                if name == m:
                    dict_num_mutation[m] += add_count  #!!!赋值不可以只写c，一定要写成键的值。
        if type2 =='del':
            add_count = len(v)

            dict_num_mutation['del'] += add_count
        if type2 =='ins':
            add_count = len(v)

            dict_num_mutation['ins'] += add_count



    # print(dict_num_mutation)

    #算比例的分母
    count_all = 0
    for k,v in dict_num_mutation.items():
        count_all += v
    # print(count_all)
    dict_proportion =  dict.fromkeys(dict_num_mutation.keys())
    for k in dict_proportion.keys():
        dict_proportion[k] = {}
        dict_proportion[k]['count'] = dict_num_mutation[k]
        if count_all!= 0:
            dict_proportion[k]['proportion'] = dict_proportion[k]['count']/count_all
        else:
            dict_proportion[k]['proportion'] = '/'
    print(dict_proportion)  
    with open(f"/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/07mutation_proportion/07snp_mutation_type_{p}.csv","w") as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerow(['mutation_type','count','proportion','group'])
        for k,v in dict_proportion.items():
            writer.writerow([k,v['count'],v['proportion'],'SNP'])

    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!统计所有的突变及其数量，频率。 后面看在哪个突变上要用。             
    with open(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/06product_exist/06snp_mutation_{p}.csv','w') as f:
        writer = csv.writer(f)
        writer.writerow(['isnv','sample','frequency','type'])
        for k,v in dict_003_08_next_generation.items():
            for s,info in v.items():
                frequency = info[0]
                type3 = info[1]
                writer.writerow([k,s.split('.')[0],frequency,type3])
            
            
            

In [ ]:
# 05 翻译蛋白质，要加3-80%的。isnv的
#分批次
#非同义氨基酸突变的具体情况和不同蛋白非同义突变的数量。
#插入加了。
from Bio.Seq import Seq
import os
import pandas as pd



wenjianjia_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/yjq_sra_bioproject_rsv')
for i in wenjianjia_list:
    dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/08product_frequency/{i}'
    if not os.path.exists(dirs):
        os.makedirs(dirs)
dict_pep_cds = {'99_518':'NS1_1C','628_1002':"NS2_1B",'1140_2315':"N",'2348_3073':"P",'3233_4003':"M",'4274_4468':"SH",'4659_5555':"G",'5632_7356':"F",'7576_8160':"M2",'8468_14965':"L"}

csv_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/06product_exist')
for c in csv_list:
    if c.endswith('csv'):
        c1 = c.split('_')[2].split('.')[0]
        csv_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/06product_exist/06isnv_mutation_{c1}.csv'
        with open(csv_path) as csvfile:
            dict_info = {}
            info_list = []
            csv_reader = csv.reader(csvfile)  
            for row in csv_reader:
                if row[0] != 'isnv':
                    mut = row[0]
                    if mut not in info_list:
                        info_list.append(mut)
        dict_info = dict.fromkeys(info_list)


        for i in dict_info.keys():#嵌套字典可以这样赋值：1、dict_info = dict.fromkeys(info_list)2、每个键都赋值成空字典，再正常加就好了
            dict_info[i] = {}


    #----------------
    #新加的，统计每个突变的样本数量。
        df = pd.read_csv(csv_path)
        df_counts = df.isnv.value_counts()
        #print(df_counts)
        outputpath=f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/06product_exist/mut_sample_number/{c1}_mut_sample_number.csv'
        df_counts.to_csv(outputpath,sep=',',index=True,header=False)

        dict_mut_num = {}
        
        mut_number_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/06product_exist/mut_sample_number/{c1}_mut_sample_number.csv'
        with open(mut_number_path) as csvfile:
            csv_reader = csv.reader(csvfile) 
            for row in csv_reader:
                isnv = row[0]
                num = row[1]
                dict_mut_num[isnv] = num
        # print(dict_mut_num)


    #--------------
        with open(csv_path) as csvfile:
            csv_reader = csv.reader(csvfile)  
            for row in csv_reader:
                if row[0] != 'isnv':
                    mut = row[0]
                    pos = int(row[0].split('_')[1])
                    fre = float(row[2])
                    type4 = row[3]
                    first = row[0].split('_')[0]
                    last = row[0].split('_')[2]
                    #分一下位置，del和ins的有可能结束位置和开始位置不在同一段蛋白里面。不太可能
                    if type4 == 'del':#按照突变前的长度算
                        pos_end = pos + len(first) -1
                    if type4 == 'ins':#按照突变后的长度算
                        pos_end = pos + len(last) -1          
                    for k,v in dict_pep_cds.items():
                        start = int(k.split('_')[0])
                        end = int(k.split('_')[1])
                        if type4 == 'snp':
                            if start <= pos <= end: #这里一定要转化成int，不然就是字符串是不对的。    
                                dict_info[mut]['product'] = v
                                dict_info[mut]['type'] = type4
                                dict_info[mut]['start'] = start
                                dict_info[mut]['end'] = end
                                dict_info[mut]['sample_number'] = dict_mut_num[mut]
        #                         dict_info[mut]['frequency'] = fre 频率先不考虑了，有好多样本的
                        if type4 == 'del' or type4 == 'ins':
                            if start <= pos <= end and start <= pos_end <= end: #这里一定要转化成int，不然就是字符串是不对的。    
                                dict_info[mut]['product'] = v
                                dict_info[mut]['type'] = type4
                                dict_info[mut]['start'] = start
                                dict_info[mut]['end'] = end
        #                         dict_info[mut]['frequency'] = fre
                                dict_info[mut]['sample_number'] = dict_mut_num[mut]




        # print(dict_info)
        ##---------------------------------------------------------------------------------------
        #去除不在cds范围内的碱基突变                   
        dict_info_feikong = {}            
        for i in dict_info.keys():
            if dict_info[i]:
                dict_info_feikong[i] = dict_info[i]
        # print(dict_info_feikong)

        #获取突变前后的三个碱基，蛋白位置不知道，可以用什么蛋白
        dict_aa_mut = {}#非同义突变的字典
        dict_aa_synonymous_mut = {}
        ref_seq = 'ACGCGAAAAAATGCGTACAACAAACTTGCGTAAACCAAAAAAATGGGGCAAATAAGAATTTGATAAGTACCACTTAAATTTAACTCCCTTGGTTAGAGATGGGCAGCAATTCATTGAGTATGATAAAAGTTAGATTACAAAATTTGTTTGACAATGATGAAGTAGCATTGTTAAAAATAACATGCTATACTGACAAATTAATACATTTAACTAATGCATTGGCTAAGGCAGTGATACATACAATCAAATTGAATGGCATTGTATTTGTGCATGTTATTACAAGTAGTGATATTTGCCCTAATAATAATATTGTAGTGAAATCCAATTTCACAACAATGCCAGTGTTACAAAATGGAGGTTATATATGGGAAATGATGGAATTAACACACTGCTCTCAACCTAATGGCCTAATAGATGACAATTGTGAAATTAAATTCTCCAAAAAACTAAGTGATTCAACAATGACCAATTATATGAATCAATTATCTGAATTACTTGGATTTGATCTTAATCCATAAATTATAATAAATATCAACTAGCAAATCAATGTCACTAACACCATTAGTTAATATAAAACTTGACAGAAGATAAAAATGGGGCAAATAAATCAATTCAGCCGACCCAACCATGGACACAACACACAATGACACCACACCACAAAGACTGATGATCACAGACATGAGACCATTGTCACTTGAGACTATAATAATATCACTAACCAGAGACATCATAACACACAGATTTATATACTTGATAAATCATGAATGTATAGTGAGAAAACTTGATGAAAGACAGGCCACATTTACATTCCTGGTCAACTATGAAATGAAACTATTGCACAAAGTGGGAAGCACTAAATACAAAAAATATACTGAATACAACACAAAATATGGCACTTTTCCTATGCCAATATTTATCAATCATGATGGGTTCTTAGAATGCATTGGCATTAAGCCTACAAAGCACACTCCCATAATATACAAGTATGATCTCAATCCATGAATTTCAACACAAGAGTCACACAATCTGAAATAACAACTTCATGCATAACCACACTCCATAGTTCAAATGGAGCCTGAAAATTATAGTAATTTAAAATTAAGGAGAGACATAAGATGAAAGATGGGGCAAATACAAAAATGGCTCTTAGCAAAGTCAAGTTGAATGATACACTCAACAAAGATCAACTTCTGTCATCCAGCAAATACACCATCCAACGGAGCACAGGAGATAGTATTGATACTCCTAATTATGATGTGCAGAAACACATCAATAAGTTATGTGGCATGTTATTAATCACAGAAGATGCTAATCATAAATTCACTGGGTTAATAGGTATGTTATATGCTATGTCTAGATTAGGAAGAGAAGACACCATAAAAATACTCAGAGATGCGGGATATCATGTAAAAGCAAATGGAGTGGATGTAACAACACATCGTCAAGATATTAATGGGAAAGAAATGAAATTTGAAGTGTTAACATTGTCAAGCTTAACAACTGAAATTCAAATCAACATTGAGATAGAATCTAGAAAATCCTACAAAAAAATGCTAAAAGAAATGGGAGAGGTAGCTCCAGAATACAGGCATGACTCTCCTGATTGTGGGATGATAATATTATGTATAGCGGCATTAGTAATAACCAAATTAGCAGCAGGGGATAGATCTGGTCTTACAGCTGTGATTAGGAGGGCTAATAATGTCCTAAAAAATGAAATGAAACGTTATAAAGGCTTACTACCCAAGGATATAGCCAACAGCTTCTATGAAGTGTTTGAAAAATATCCTCACTTTATAGATGTTTTTGTTCATTTTGGTATAGCACAATCTTCTACCAGAGGTGGCAGTAGAGTTGAAGGGATTTTTGCTGGATTGTTTATGAATGCCTATGGTGCAGGGCAAGTGATGTTACGGTGGGGGGTCTTAGCAAAATCAGTTAAAAATATTATGCTAGGACACGCTAGTGTGCAAGCAGAAATGGAACAAGTTGTGGAGGTTTATGAATATGCCCAAAAATTGGGTGGAGAAGCAGGGTTCTACCATATATTGAACAACCCAAAAGCATCATTATTGTCTTTGACTCAATTTCCTCACTTCTCCAGTGTAGTATTAGGCAATGCTGCTGGCCTAGGCATAATGGGAGAATACAGAGGTACACCAAGGAATCAAGATCTATATGATGCTGCAAAAGCATATGCTGAACAACTCAAAGAAAATGGTGTGATTAACTACAGTGTATTAGACTTGACAGCAGAAGAACTAGAGGCTATCAAACATCAGCTTAATCCAAAAGATAATGATGTAGAGCTTTGAGTTAATAAAAAAAATGGGGCAAATAAAACATCATGGAAAAGTTTGCTCCTGAATTCCATGGAGAAGATGCAAACAACAGAGCTACCAAATTCCTAGAATCAATAAAGGGCAAATTCACATCACCTAAAGATCCCAAGAAAAAAGATAGTATCATATCTGTCAACTCAATAGATATAGAAGTAACCAAAGAAAGCCCTATAACATCAAATTCAACCATTATAAACCCAACAAATGAGACAGATGATACTGTAGGGAACAAGCCCAATTATCAAAGAAAACCTCTAGTAAGTTTCAAAGAAGACCCTACGCCAAGTGATAATCCCTTTTCAAAACTATACAAAGAAACCATAGAAACATTTGATAACAATGAAGAAGAATCTAGCTATTCATATGAAGAAATAAATGATCAGACAAACGATAATATAACAGCAAGATTAGATAGGATTGATGAAAAATTAAGTGAAATACTAGGAATGCTTCACACATTAGTAGTAGCGAGTGCAGGACCTACATCTGCTCGGGATGGTATAAGAGATGCCATGGTTGGTTTAAGAGAAGACATGATAGAAAAAATCAGAACTGAAGCATTAATGACCAATGACAGACTAGAAGCTATGGCAAGACTCAGGAATGAGGAAAGTGAAAAGATGGCAAAAGACACATCAGATGAAGTGTCTCTCAATCCAACATCAGAGAAATTGAACAACCTGTTGGAAGGGAATGATAGTGACAATGATCTATCACTTGATGATTTCTGATCAGTTACCAATCTGTACATCAACACACAACACCAACAGAAGACCAACAAACAAACCAACTCACCCATCCAACCAAACATCTATACGCCAATCAGCCAATCCAAAACTAGCCACCCGGAAAAAATAGATACTATAGTTACAAAAAAAGATGGGGCAAATATGGAAACATACGTGAACAAACTTCACGAAGGCTCCACATACACAGCTGCTGTTCAATACAATGTCTTAGAAAAAGACGATGACCCTGCATCACTTACAATATGGGTGCCCATGTTCCAATCATCCATGCCAGCAGATTTACTTATAAAAGAACTAGCTAATGTCAACATACTAGTGAAACAAATATCCACACCCAATGGACCTTCATTAAGAGTCATGATAAACTCAAGAAGTGCAGTGCTAGCACAAATGCCCAGCAAATTTACCATATGTGCCAATGTGTCCTTGGATGAAAGAAGCAAGCTGGCATATGATGTAACCACACCCTGTGAAATCAAGGCATGTAGTCTAACATGCCTAAAATCAAAAAATATGTTAACTACAGTTAAAGATCTCACTATGAAAACACTCAACCCAACACATGACATCATTGCTTTATGTGAATTTGAAAATATAGTAACATCAAAAAAAGTCATAATACCAACATACCTAAGATCCATCAGTGTCAGAAATAAAGATCTGAACACACTTGAAAATATAACAACCACTGAATTCAAAAATGCCATCACAAATGCAAAAATCATCCCTTACTCAGGATTACTGTTAGTCATCACAGTGACTGACAACAAAGGAGCATTCAAATACATAAAGCCACAAAGTCAATTTATAGTAGATCTTGGAGCTTACCTAGAAAAAGAAAGTATATATTATGTTACAACAAATTGGAAGCACACAGCTACACGATTTGCAATCAAACCCATGGAAGATTAACCTTTTTCTTCTACATCAGTGAGTTGATTCATACAAACTTTCTACCTACATTCTTCACTTCACCATCATAATCACCAACCCTCTGTGGTTCAACTAATCAAACAAAACCCATCTGGAGCCTCAGATCATCCCAAGTCATTGTTCATCAGATCTAGTACTCAAATAAGTTAATAAAAATATCCACATGGGGCAAATAATCATTGGAGGAAATCCAACTAATCACAATATCTGTCAACATAGACAAGTCAACACGCCAGGCAAAATCAACCAATGGAAAATACATCCATAACAATAGAATTCTCAAGCAAATTCTGGCCTTACTTTACACTAATACACATGATAACAACAATAATCTCTTTGCTAATCATAATCTCCATCATGATTGCAATACTGAACAAACTCTGTGAATATAACGTATTCCATAACAAAACCTTTGAGCTACCAAGAGCTCGAGTCAATACATAGCATTCACCAATCTGATGGCACAAAACAGTAACCTTGCATTTGTAAGTGAACAACCCTCACCTCTTTACAAAACCACATCAACATCTCACCATGCAAGCCATCATCCATATTATAAAGTAGTTAATTAAAAATAATCATAACAATGAACTAAGATATTAAGACTAACAATAACGTTGGGGCAAATGCAAACATGTCCAAAAACAAGGACCAACGCACCACCAAGACACTAGAAAAGACCTGGGACACTCTCAATCATCTATTATTCATATCATCGTGCTTATACAAGTTAAATCTTAAATCTATAGCACAAATCACATTATCCATTCTGGCAATGATAATCTCAACTTCACTTATAATTGCAGCCATCATATTCATAGCCTCGGCAAACCACAAAGTCACACTAACAACTGCAATCATACAAGATGCAACAAGCCAGATCAAGAACACAACCCCAACATACCTCACCCAGAATCCCCAGCTTGGAATCAGCTTCTCCAATCTGTCTGAAACTACATCACAAACCACCACCATACTAGCTTCAACAACACCAAGTGTCAAGTCAACCCTGCAATCCACAACAGTCAAGACCAAAAACACAACAACAACCAAAATACAACCCAGCAAGCCCACCACAAAACAACGCCAAAACAAACCACCAAACAAACCCAATAATGATTTTCACTTTGAAGTGTTCAACTTTGTACCTTGCAGCATATGCAGCAACAATCCAACCTGCTGGGCTATCTGTAAAAGAATACCAAACAAAAAACCTGGAAAGAAAACCACCACCAAGCCCACAAAAAAACCAACCATCAAGACAACCAAAAAAGATCTCAAACCTCAAACCACAAAACCAAAGGAAGTACCTACCACCAAGCCCACAGAAAAGCCAACCATCAACACCACCAAAACAAACATCAGAACTACACTGCTCACCAACAATACCACAGGAAATCCAGAACACACAAGTCAAAAGGGAACCCTCCACTCAACCTCCTCCGATGGCAATCCAAGCCCTTCACAAGTCTATACAACATCCGAGTACCTATCACAACCTCCATCTCCATCCAACACAACAAACCAGTAGTCATTAAAAAGCGTATTATTGCAAAAAGCCATGACCAAATCAACCAGAATCAAAATCAACTCTGGGGCAAATAACAATGGAGTTGCCAATCCTCAAAACAAATGCAATTACCGCAATCCTTGCTGCAGTCACACTCTGTTTTGCTTCCAGTCAAAACATCACTGAAGAATTTTATCAAACAACATGCAGTGCAGTCAGCAAAGGCTATCTTAGTGCTCTAAGAACTGGTTGGTATACTAGTGTTATAACTATAGAATTAAGTAATATCAAGGAAAATAAGTGTAATGGAACAGACGCTAAGGTAAAATTGATAAAACAAGAATTAGATAAATATAAAAGTGCTGTAACAGAATTGCAGTTGCTCATGCAAAGCACACCGGCAACCAACAATCGAGCCAGAAGAGAACTACCAAGGTTTATGAATTATACACTCAACAATACCAAAAATACCAATGTAACATTAAGCAAGAAAAGGAAAAGAAGATTTCTTGGCTTTTTGTTAGGTGTTGGATCTGCAATCGCCAGTGGCATTGCTGTATCTAAGGTCCTGCACCTAGAAGGGGAAGTGAACAAAATCAAAAGTGCTCTACTATCCACAAACAAGGCTGTAGTCAGCTTATCAAATGGAGTTAGTGTCTTAACCAGCAAAGTGTTAGACCTCAAAAACTATATAGATAAACAGTTGTTACCTATTGTGAACAAGCAAAGCTGTAGCATATCAAACATTGAAACTGTGATAGAGTTCCAACAAAAGAACAACAGACTACTAGAGATTACCAGGGAATTTAGTGTTAATGCAGGTGTAACTACACCTGTAAGCACTTATATGTTAACAAATAGTGAATTATTATCATTAATCAATGATATGCCTATAACAAATGATCAGAAAAAGTTAATGTCCAACAATGTTCAAATAGTTAGACAGCAAAGTTACTCTATCATGTCCATAATAAAGGAGGAAGTCTTAGCATATGTAGTACAATTACCACTATATGGTGTAATAGATACACCTTGTTGGAAACTGCACACATCCCCTCTATGTACAACCAACACAAAGGAAGGGTCCAACATCTGTTTAACAAGAACCGACAGAGGATGGTACTGTGACAATGCAGGATCAGTATCTTTCTTCCCACTAGCTGAAACATGTAAAGTTCAATCGAATCGAGTATTTTGTGACACAATGAACAGTTTAACATTACCAAGTGAAGTAAATCTCTGCAACATTGACATATTCAACCCCAAATATGATTGCAAAATTATGACTTCAAAAACAGATGTAAGCAGCTCCGTTATCACATCTCTAGGAGCCATTGTGTCATGCTATGGCAAAACTAAATGTACAGCATCCAATAAAAATCGTGGAATCATAAAGACATTTTCTAACGGGTGCGATTATGTATCAAATAAGGGGGTTGACACTGTGTCTGTAGGTAATACATTATATTATGTAAATAAGCAAGAAGGCAAAAGTCTCTATGTAAAAGGTGAACCAATAATAAATTTCTATGACCCATTAGTGTTCCCCTCTGATGAATTTGATGCATCAATATCTCAAGTCAATGAGAAGATTAACCAGAGCCTAGCATTTATTCGTAAATCCGATGAATTATTACATAATGTAAATGCTGGTAAATCCACCATAAATATCATGATAACTACTATAATTATAGTGATTATAGTAATATTGTTATCATTAATTGCCGTTGGACTGCTCCTATACTGCAAGGCCAGAAGCACACCAGTCACACTAAGCAAGGATCAACTGAGTGGTATAAATAATATTGCATTTAGTAACTAAATAAAAATAGCACCTAATCATGTTCTTACAATGGTTTCATATCTGCTCATAGACAACCCATCTATCATTGGATTTTCTTAAAATCTGAACTTCATCGAAACTCTCATCTATAAACCATCTCACTTACATTATTTAAGTAGATTCCTAGTTTATAGTTATATAAAACAATTGAATACCAGATTAACTTACTATTTGTAAAAAATGAGAACTGGGGCAAATATGTCACGAAGGAATCCTTGCAAATTTGAAATTCGAGGTCATTGCTTGAATGGTAAGAGGTGTCATTTTAGTCATAATTATTTTGAATGGCCACCCCATGCACTGCTTGTAAGACAAAACTTTATGTTAAACAGAATACTTAAGTCTATGGATAAAAGCATCGATACTTTATCAGAAATAAGTGGAGCTGCAGAGTTGGACAGAACAGAAGAGTATGCCCTCGGTGTAGTTGGAGTGCTAGAGAGTTATATAGGATCTATAAATAATATAACTAAACAATCAGCATGTGTTGCCATGAGCAAACTCCTCACTGAACTCAACAGTGATGACATCAAAAAACTGAGGGACAATGAAGAGCCAAATTCACCCAAGATAAGAGTGTACAATACTGTCATATCATATATTGAAAGCAACAGGAAAAACAATAAACAAACTATCCATCTGTTAAAAAGATTGCCAGCAGACGTATTGAAGAAAACCATAAAAACCACATTGGATATCCACAAGAGCATAACCATCAATAACCCAAAAGAATCAACTGTTAGTGATATAAACGACCATGCCAAAAATAATGATACTACCTGACAAATATCCTTGTAGTATAAATTCCATACTAATAACAAGTAGTTGTAGAGTTACTATGTATAATCAAAAGAACACACTATATTTCAATCAAAACAACCAAAATAACCATATATACTCACCGAATCAACCATTCAATGAAATCCATTGGACCTCTCAAGACTTGATTGATGCAATTCAAAATTTTCTACAACATCTAGGTATTACTGATGATATATACACAATATATATATTAGTGTCATAACACTCAATCCTAATGCTTACCACATCATCAAACTATTAACTCAAACAATTCAAGCCATGGGACAAAATGGATCCCATTATTAATGGAAATTCTGCTAATGTGTATCTAACCGATAGTTATTTAAAAGGTGTTATTTCTTTCTCAGAATGTAATGCTTTAGGAAGTTACATATTCAATGGTCCTTATCTCAAAAATGATTATACCAACTTAATTAGTAGACAAAATCCATTAATAGAACACATAAATCTAAAGAAACTAAATATAACACAGTCCTTAATGTCTAAGTATCATAAAGGTGAAATAAAAATAGAAGAACCTACTTATTTTCAGTCATTACTTATGACATACAAGAGTATGACCTCGTTAGAACAGATTACTACCACTAATTTACTTAAAAAGATAATAAGAAGAGCTATAGAAATTAGTGATGTCAAAGTCTATGCTATATTGAATAAACTGGGGCTTAAAGAAAAAGACAAGATTAAATCCAACAATGGACAAGATGAAGACAACTCAGTTATTACAACCATAATCAAAGATGATATACTTTTAGCTGTTAAGGATAATCAATCTCATCTTAAAGCAGTCAAAAATCACTCTACAAAACAAAAAGATACAATCAAAACAACACTCTTGAAGAAATTAATGTGTTCAATGCAACATCCTCCATCATGGTTAATACATTGGTTTAATTTATACACAAAATTAAACAACATATTAACACAGTATCGATCAAGTGAGGTAAAAAACCATGGTTTTATATTGATAGACAATCATACTCTCAATGGATTCCAATTTATTTTGAATCAATATGGTTGTATAGTTTATCATAAGGAACTCAAAAGAATTACTGTGACAACCTATAATCAATTCTTGACATGGAAAAATATTAGCCTTAGTAGATTAAATGTTTGTTTAATTACATGGATTAGTAACTGTTTGAACACATTAAATAAAAGCTTAGGCTTAAGATGCGGATTCAATAATGTTATCTTGACACAACTATTCCTCTATGGAGATTGTATACTAAAACTATTCCACAATGAGGGGTTCTACATAATAAAAGAGGTAGAGGGATTTATTATGTCTCTAATTTTAAATATAACAGAAGAAGATCAATTCAGAAAACGGTTTTATAATAGTATGCTCAACAACATCACAGATGCTGCTAATAAAGCTCAGAAAAATCTGCTATCAAGAGTATGTCATACATTATTAGATAAGACAGTATCCGATAATATAATAAATGGCAGATGGATAATTCTATTAAGTAAGTTCCTTAAATTAATTAAGCTTGCAGGTGACAATAACCTTAACAATCTGAGTGAATTATATTTTTTGTTCAGAATATTTGGACACCCAATGGTAGATGAAAGACAAGCCATGGATGCTGTTAAAGTTAATTGCAACGAGACCAAATTTTACTTGTTAAGCAGTTTGAGTATGTTAAGAGGTGCCTTTATATATAGAATTATAAAAGGATTTGTAAATAATTACAACAGATGGCCTACTTTAAGGAATGCTATTGTTTTACCCTTAAGATGGTTAACTTACTATAAACTAAACACTTATCCTTCCTTGTTGGAACTTACAGAAAGAGATTTGATTGTTTTATCAGGACTACGTTTCTATCGTGAGTTTCGGTTGCCTAAAAAAGTGGATCTTGAAATGATCATAAATGATAAGGCTATATCACCTCCTAAAAATTTGATATGGACTAGTTTCCCTAGAAATTATATGCCGTCACACATACAAAATTATATAGAACATGAAAAATTAAAATTTTCCGAGAGTGATAAATCAAGAAGAGTATTAGAGTACTATTTAAGAGATAACAAATTCAATGAATGTGATTTATACAACTGTGTAGTTAATCAAAGTTATCTTAACAACCCTAATCATGTGGTATCTTTGACAGGCAAAGAAAGAGAACTCAGTGTAGGTAGAATGTTTGCAATGCAACCAGGAATGTTCAGACAAGTTCAAATATTAGCAGAGAAAATGATAGCTGAAAACATTTTACAATTCTTTCCTGAAAGTCTTACAAGATATGGTGATCTAGAACTACAGAAAATATTAGAATTGAAAGCAGGAATAAGTAACAAATCAAATCGTTACAATGATAATTACAACAATTACATTAGTAAGTGCTCTATCATCACAGATCTCAGCAAATTCAATCAAGCATTTCGATATGAAACATCATGTATTTGTAGTGATGTACTGGATGAACTGCATGGTGTACAATCTCTATTTTTCTGGTTACATTTAGCTATTCCTCATGTCACAATAATATGCACATATAGGCATGCACCCCCCTATATAAGAGATCATATTGTAGATCTTAACAATGTAGATGAACAAAGTGGATTATATAGATATCATATGGGTGGTATCGAAGGGTGGTGTCAAAAACTATGGACCATAGAAGCTATATCACTATTGGATCTAATATCTCTCAAAGGGAAATTCTCAATTACTGCTTTAATTAATGGTGACAATCAATCAATAGATATAAGTAAACCAGTCAGACTCATGGAAGGTCAAACTCATGCTCAAGCAGATTATTTGCTAGCATTAAATAGTCTTAAATTACTGTATAAAGAGTATGCAGGCATAGGCCACAAATTAAAAGGAACTGAGACTTATATATCAAGAGATATGCAATTTATGAGTAAAACAATTCAACATAACGGTGTATATTACCCAGCTAGTATAAAGAAAGTCCTAAGAGTGGGACCGTGGATAAACACTATACTTGATGATTTCAAAGTGAGTCTAGAATCTATAGGTAGTTTGACACAAGAATTAGAATATAGAGGAGAAAGTCTATTATGCAGTTTAATATTTAGAAATGTATGGTTATATAATCAAATTGCTTTACAACTAAAAAATCATGCATTATGTAACAATAAATTATATTTGGACATATTAAAGGTTCTGAAACACTTAAAAACCTTTTTTAATCTTGATAATATTGATACAGCATTAACATTGTATATGAATTTGCCCATGTTATTTGGTGGTGGTGATCCCAACTTGTTATATCGAAGTTTCTATAGAAGAACTCCTGATTTCCTCACAGAGGCTATAGTTCACTCTGTGTTCATACTTAGTTATTATACAAACCATGATTTAAAAGATAAACTTCAAGATCTGTCAGATGATAGATTGAATAAGTTCTTAACATGCATAATCACATTTGACAAAAACCCTAATGCTGAATTCGTAACATTGATGAGAGATCCTCAAGCTTTAGGGTCTGAGAGACAAGCTAAAATTACTAGCGAAATCAATAGACTGGCAGTTACTGAGGTTTTGAGCACAGCTCCAAACAAAATATTCTCCAAAAGTGCACAACACTATACCACTACAGAGATAGATCTAAATGATATTATGCAAAATATAGAACCTACATATCCTCACGGGCTAAGAGTTGTTTATGAAAGTTTACCCTTTTATAAAGCAGAGAAAATAGTAAATCTTATATCCGGTACAAAATCTATAACTAACATACTGGAAAAGACTTCTGCCATAGACTTAACAGATATTGATAGAGCCACTGAGATGATGAGGAAAAACATAACTTTGCTTATAAGGATATTTCCATTAGATTGTAACAGAGATAAAAGGGAAATATTGAGTATGGAAAACCTAAGTATTACTGAATTAAGCAAATATGTTAGGGAAAGATCTTGGTCTTTATCCAATATAGTTGGTGTTACATCACCTAGTATCATGTATACAATGGACATCAAATATACAACAAGCACTATAGCTAGTGGCATAATCATAGAGAAATATAATGTTAACAGTTTAACACGTGGTGAGAGAGGACCCACTAAACCATGGGTTGGTTCATCTACACAAGAGAAAAAAACAATGCCAGTTTATAATAGACAAGTTTTAACCAAAAAACAGAGAGATCAAATAGATCTATTAGCAAAATTGGATTGGGTGTATGCATCTATAGATAACAAGGATGAATTCATGGAAGAACTTAGCATAGGAATTCTTGGGTTAACATATGAGAAAGCCAAAAAATTATTTCCACAATATTTAAGTGTTAACTATTTGCATCGCCTTACAGTCAGTAGTAGACCATGTGAATTCCCTGCATCAATACCAGCTTATAGAACTACAAATTATCACTTTGATACTAGCCCTATTAATCGCATATTAACAGAAAAGTATGGTGATGAAGATATTGATATAGTATTCCAAAACTGTATAAGCTTTGGCCTTAGCTTAATGTCAGTAGTAGAACAATTTACTAATGTATGTCCTAACAGAATTATTCTTATACCTAAGCTTAATGAGATACATTTAATGAAACCTCCCATATTCACAGGTGATGTTGATATTCACAAGTTAAAACAAGTGATACAAAAACAGCATATGTTTTTACCAGACAAAATAAGTTTGACTCAATATGTGGAATTATTCTTAAGTAATAAAACACTCAAATCTGGATCTCATGTTAATTCTAATTTAATATTGGCGCATAAGATATCTGACTATTTTCATAATACTTACATTTTAAGTACTAATTTAGCTGGACATTGGATTCTGATTATACAACTTATGAAAGATTCTAAGGGTATTTTTGAAAAAGATTGGGGAGAGGGATATATAACTGATCATATGTTCATTAATTTGAAAGTTTTCTTCAATGCTTATAAGACCTATCTCTTGTGTTTTCATAAAGGTTACGGCAGAGCAAAGCTGGAGTGTGATATGAATACTTCAGATCTCCTATGTGTATTGGAATTAATAGACAGTAGTTATTGGAAGTCTATGTCTAAGGTATTTTTAGAACAAAAAGTTATCAAATACATTCTTAGCCAGGATGCAAGTTTACATAGAGTAAAAGGATGTCATAGCTTCAAACTATGGTTTCTTAAACGTCTTAATGTAGCAGAATTCACAGTTTGCCCTTGGGTTGTTAACATAGATTATCATCCAACACATATGAAAGCAATATTAACTTATATAGATCTTGTTAGAATGGGATTGATAAATATAGATAAAATATACATTAAAAATAAACACAAATTCAATGATGAATTTTATACTTCTAATCTCTTTTACATTAATTATAACTTCTCAGATAATACTCATCTATTAACTAAACATATAAGGATTGCTAATTCTGAATTAGAAAATAATTACAACAAATTATATCATCCTACACCAGAAACCCTAGAAAATATACTAACCAATCCGGTTAAATGTAATGACAAAAAGACACTGAATGACTATTGTATAGGTAAAAATGTTGACTCAATAATGTTACCATTGTTATCTAATAAGAAGCTTATTAAATCGCCTACAATGATTAGAACCAATTACAGCAAACAAGATTTGTATAATTTATTTCCTACGGTTGTGATTGATAAAATTATAGATCATTCAGGTAATACAGCCAAATCTAACCAACTTTACACTACTACTTCTCATCAAATACCTTTAGTGCACAATAGCACATCACTTTATTGCATGCTTCCTTGGCATCATATTAATAGATTCAATTTTGTATTTAGTTCTACAGGTTGTAAAATTAGTATAGAGTATATTTTAAAAGACCTTATAATTAAAGATCCTAATTGTATAGCATTCATAGGTGAAGGAGCAGGGAATTTATTATTGCGTACAGTAGTGGAACTTCATCCCGATATAAGATATATTTACAGAAGTCTGAAGGATTGCAATGATCATAGTTTACCTATTGAGTTTTTAAGGCTGTACAATGGACATATCAACATTGATTATGGTGAAAATTTGACCATTCCTGCTACAGATGCAACCAACAACATTCATTGGTCTTATTTACATATAAAGTTTGCTGAACCTATCAGTCTTTTTGTCTGTGATGCTGAATTGCCTGTAACAGTCAACTGGAGTAAAATTATAATAGAGTGGAGCAAGCATGTAAGAAAATGCAAGTACTGTTCCTCAGTTAATAAATGTACGTTAATAGTAAAATATCATGCTCAAGATGATATCGATTTCAAATTAGACAATATAACTATATTAAAAACTTATGTATGCTTAGGCAGTAAGTTAAAGGGGTCTGAAGTTTACTTAGTCCTTACAATAGGTCCTGCAAATGTGTTCCCAGTATTTAATGTAGTACAAAATGCTAAATTGATACTATCAAGAACCAAAAATTTCATCATGCCTAAGAAGGCTGATAAAGAGTCTATTGATGCAAATATTAAAAGTTTGATACCCTTTCTTTGTTACCCTATAACAAAAAAAGGAATTAATACTGCATTGTCAAAACTAAAGAGTGTTGTTAGTGGAGATATACTATCATATTCTATAGCAGGACGTAATGAAGTTTTCAGCAATAAACTTATAAATCATAAGCATATGAACATCTTAAAATGGTTCAATCATGTTTTAAATTTCAGATCAACAGAACTAAACTATAATCATTTATATATGGTAGAATCTACATATCCTTATCTAAGTGAATTGTTAAACAGCTTGACAACTAATGAACTTAAAAAACTGATTAAAATCACAGGTAGTTTGTTATACAACTTTCATAATGAATAATGAATAAAAATCTTATATTAAAAATTCCCATAGCTACACACTAACACTGTATTCAATTATAGTTATTTAAAATTAAAAATTATATAATTTTTTAATAACTTTTAGTGAACTAATCCTAAAATTATCATTTTGATCTAGGAGGAATAAATTTAAATCCAAATCTAATTGGTTTATATGTATATTAACTAAACTACGAGATATTAGTTTTTGACACTTTTTTTCTCGT'
        ref_seq_list = list(ref_seq)
        dict_jianji = {}
        for k,v in dict_info_feikong.items():
            gene_pos = int(k.split('_')[1])
            ref = k.split('_')[0]
            alt = k.split('_')[2]
            product = v['product']
        #     frequency = v['frequency']
            gene_pos_f2 = gene_pos - 2   #前面2个
            gene_pos_f1 = gene_pos - 1   #前面1个
            gene_pos_b1 = gene_pos + 1
            gene_pos_b2 = gene_pos + 2
            start = v['start']
            end = v['end']
            type4 = v['type']
            chazhi = int(gene_pos) - int(start)
            quzheng =  chazhi//3   #取整，以获取是这个蛋白质的第几个氨基酸。
            aa_pos = quzheng + 1

        # #分类讨论
            if type4 == 'del' or type4 == 'ins':
                del_count = len(ref) - len(alt)
                if del_count%3 == 0:
                    #分类讨论
                    #1
                    if chazhi%3 == 2 and len(alt)%3 == 2:
                        #变异前的位置和碱基
                        list_jianji_pos_ref = []#是位置不是切片，切片要-1
                        list_jianji_ref = []
                        list_jianji_pos_ref.append(gene_pos_f2)
                        list_jianji_pos_ref.append(gene_pos_f1)
                        for index in range(0,len(ref)):
                            list_jianji_pos_ref.append((gene_pos + index))
                        list_jianji_pos_ref.append(gene_pos_b1)
                        list_jianji_pos_ref.append(gene_pos_b2)
                        for p in list_jianji_pos_ref:
                            p_qiepian = p-1
                            list_jianji_ref.append(ref_seq[p_qiepian])


                        #变异后的位置
                        list_jianji_pos_alt = []#是位置不是切片，切片要-1
                        list_jianji_alt = []
                        list_jianji_pos_alt.append(gene_pos_f2)
                        list_jianji_pos_alt.append(gene_pos_f1)

                        list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                        list_jianji_alt.append(ref_seq[gene_pos_f1-1])
                        for index in range(0,len(alt)):
                            list_jianji_pos_alt.append((gene_pos + index))
                            list_jianji_alt.append(alt[index])
                        list_jianji_pos_alt.append(gene_pos_b1)
                        list_jianji_pos_alt.append(gene_pos_b2)
                        list_jianji_alt.append(ref_seq[gene_pos_b1-1])
                        list_jianji_alt.append(ref_seq[gene_pos_b2-1])
                   #2
                    if chazhi%3 == 2 and len(alt)%3 == 1:
                        list_jianji_pos_ref = []#是位置不是切片，切片要-1
                        list_jianji_ref = []
                        list_jianji_pos_ref.append(gene_pos_f2)
                        list_jianji_pos_ref.append(gene_pos_f1)
                        for index in range(0,len(ref)):
                            list_jianji_pos_ref.append((gene_pos + index))

                        for p in list_jianji_pos_ref:
                            p_qiepian = p-1
                            list_jianji_ref.append(ref_seq[p_qiepian])


                        #变异后的位置
                        list_jianji_pos_alt = []#是位置不是切片，切片要-1
                        list_jianji_alt = []
                        list_jianji_pos_alt.append(gene_pos_f2)
                        list_jianji_pos_alt.append(gene_pos_f1)

                        list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                        list_jianji_alt.append(ref_seq[gene_pos_f1-1])
                        for index in range(0,len(alt)):
                            list_jianji_pos_alt.append((gene_pos + index))
                            list_jianji_alt.append(alt[index])

                    #3
                    if chazhi%3 == 2 and len(alt)%3 == 0:
                    #变异前的位置和碱基
                        list_jianji_pos_ref = []#是位置不是切片，切片要-1
                        list_jianji_ref = []
                        list_jianji_pos_ref.append(gene_pos_f2)
                        list_jianji_pos_ref.append(gene_pos_f1)
                        for index in range(0,len(ref)):
                            list_jianji_pos_ref.append((gene_pos + index))
                        list_jianji_pos_ref.append(gene_pos_b1)

                        for p in list_jianji_pos_ref:
                            p_qiepian = p-1
                            list_jianji_ref.append(ref_seq[p_qiepian])


                        #变异后的位置
                        list_jianji_pos_alt = []#是位置不是切片，切片要-1
                        list_jianji_alt = []
                        list_jianji_pos_alt.append(gene_pos_f2)
                        list_jianji_pos_alt.append(gene_pos_f1)

                        list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                        list_jianji_alt.append(ref_seq[gene_pos_f1-1])
                        for index in range(0,len(alt)):
                            list_jianji_pos_alt.append((gene_pos + index))
                            list_jianji_alt.append(alt[index])
                        list_jianji_pos_alt.append(gene_pos_b1)
                        list_jianji_alt.append(ref_seq[gene_pos_b1-1])

                    #4
                    if chazhi%3 == 1 and len(alt)%3 == 2:
                        #变异前的位置和碱基
                        list_jianji_pos_ref = []#是位置不是切片，切片要-1
                        list_jianji_ref = []
                        list_jianji_pos_ref.append(gene_pos_f2)

                        for index in range(0,len(ref)):
                            list_jianji_pos_ref.append((gene_pos + index))

                        for p in list_jianji_pos_ref:
                            p_qiepian = p-1
                            list_jianji_ref.append(ref_seq[p_qiepian])


                        #变异后的位置
                        list_jianji_pos_alt = []#是位置不是切片，切片要-1
                        list_jianji_alt = []
                        list_jianji_pos_alt.append(gene_pos_f2)


                        list_jianji_alt.append(ref_seq[gene_pos_f2-1])

                        for index in range(0,len(alt)):
                            list_jianji_pos_alt.append((gene_pos + index))
                            list_jianji_alt.append(alt[index])

                    #5
                    if chazhi%3 == 1 and len(alt)%3 == 1:
                        #变异前的位置和碱基
                        list_jianji_pos_ref = []#是位置不是切片，切片要-1
                        list_jianji_ref = []
                        list_jianji_pos_ref.append(gene_pos_f2)
                        for index in range(0,len(ref)):
                            list_jianji_pos_ref.append((gene_pos + index))
                        list_jianji_pos_ref.append(gene_pos_b1)
                        for p in list_jianji_pos_ref:
                            p_qiepian = p-1
                            list_jianji_ref.append(ref_seq[p_qiepian])


                        #变异后的位置
                        list_jianji_pos_alt = []#是位置不是切片，切片要-1
                        list_jianji_alt = []
                        list_jianji_pos_alt.append(gene_pos_f2)
                        list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                        for index in range(0,len(alt)):
                            list_jianji_pos_alt.append((gene_pos + index))
                            list_jianji_alt.append(alt[index])
                        list_jianji_pos_alt.append(gene_pos_b1)
                        list_jianji_alt.append(ref_seq[gene_pos_b1-1])
                #6
                    if chazhi%3 == 1 and len(alt)%3 == 0:
                        #变异前的位置和碱基
                        list_jianji_pos_ref = []#是位置不是切片，切片要-1
                        list_jianji_ref = []
                        list_jianji_pos_ref.append(gene_pos_f2)

                        for index in range(0,len(ref)):
                            list_jianji_pos_ref.append((gene_pos + index))
                        list_jianji_pos_ref.append(gene_pos_b1)
                        list_jianji_pos_ref.append(gene_pos_b2)
                        for p in list_jianji_pos_ref:
                            p_qiepian = p-1
                            list_jianji_ref.append(ref_seq[p_qiepian])


                        #变异后的位置
                        list_jianji_pos_alt = []#是位置不是切片，切片要-1
                        list_jianji_alt = []
                        list_jianji_pos_alt.append(gene_pos_f2)


                        list_jianji_alt.append(ref_seq[gene_pos_f2-1])

                        for index in range(0,len(alt)):
                            list_jianji_pos_alt.append((gene_pos + index))
                            list_jianji_alt.append(alt[index])
                        list_jianji_pos_alt.append(gene_pos_b1)
                        list_jianji_pos_alt.append(gene_pos_b2)
                        list_jianji_alt.append(ref_seq[gene_pos_b1-1])
                        list_jianji_alt.append(ref_seq[gene_pos_b2-1])

                    #7
                    if chazhi%3 == 0 and len(alt)%3 == 2:
                        #变异前的位置和碱基
                        list_jianji_pos_ref = []#是位置不是切片，切片要-1
                        list_jianji_ref = []

                        for index in range(0,len(ref)):
                            list_jianji_pos_ref.append((gene_pos + index))
                        list_jianji_pos_ref.append(gene_pos_b1)

                        for p in list_jianji_pos_ref:
                            p_qiepian = p-1
                            list_jianji_ref.append(ref_seq[p_qiepian]) 
                        #变异后的位置
                        list_jianji_pos_alt = []#是位置不是切片，切片要-1
                        list_jianji_alt = []
                        for index in range(0,len(alt)):
                            list_jianji_pos_alt.append((gene_pos + index))
                            list_jianji_alt.append(alt[index])
                        list_jianji_pos_alt.append(gene_pos_b1)
                        list_jianji_alt.append(ref_seq[gene_pos_b1-1])

                    #8
                    if chazhi%3 == 0 and len(alt)%3 == 1:
                        #变异前的位置和碱基
                        list_jianji_pos_ref = []#是位置不是切片，切片要-1
                        list_jianji_ref = []
                        for index in range(0,len(ref)):
                            list_jianji_pos_ref.append((gene_pos + index))
                        list_jianji_pos_ref.append(gene_pos_b1)
                        list_jianji_pos_ref.append(gene_pos_b2)
                        for p in list_jianji_pos_ref:
                            p_qiepian = p-1
                            list_jianji_ref.append(ref_seq[p_qiepian])


                        #变异后的位置
                        list_jianji_pos_alt = []#是位置不是切片，切片要-1
                        list_jianji_alt = []
                        for index in range(0,len(alt)):
                            list_jianji_pos_alt.append((gene_pos + index))
                            list_jianji_alt.append(alt[index])
                        list_jianji_pos_alt.append(gene_pos_b1)
                        list_jianji_pos_alt.append(gene_pos_b2)
                        list_jianji_alt.append(ref_seq[gene_pos_b1-1])
                        list_jianji_alt.append(ref_seq[gene_pos_b2-1])
                #9
                    if chazhi%3 == 0 and len(alt)%3 == 0:
                        #变异前的位置和碱基
                        list_jianji_pos_ref = []#是位置不是切片，切片要-1
                        list_jianji_ref = []
                        for index in range(0,len(ref)):
                            list_jianji_pos_ref.append((gene_pos + index))

                        for p in list_jianji_pos_ref:
                            p_qiepian = p-1
                            list_jianji_ref.append(ref_seq[p_qiepian])

                        #变异后的位置
                        list_jianji_pos_alt = []#是位置不是切片，切片要-1
                        list_jianji_alt = []
                        for index in range(0,len(alt)):
                            list_jianji_pos_alt.append((gene_pos + index))
                            list_jianji_alt.append(alt[index])



          #------------------------原来的，底下可以不变  
            if type4 == 'snp':
                if chazhi%3 == 2:
                    list_jianji_pos = []#是位置不是切片，切片要-1
                    list_jianji_pos.append(gene_pos_f2)
                    list_jianji_pos.append(gene_pos_f1)
                    list_jianji_pos.append(gene_pos)



                if chazhi%3 == 1:
                    list_jianji_pos = []
                    list_jianji_pos.append(gene_pos_f1)
                    list_jianji_pos.append(gene_pos)
                    list_jianji_pos.append(gene_pos_b1)



                if chazhi%3 == 0:
                    list_jianji_pos = []
                    list_jianji_pos.append(gene_pos)
                    list_jianji_pos.append(gene_pos_b1)
                    list_jianji_pos.append(gene_pos_b2)


                list_jianji_qiepian = []#这个是切片
                for i in list_jianji_pos:
                    i_1 = i-1
                    list_jianji_qiepian.append(i_1)
            #     print(list_jianji_qiepian)

                list_jianji_ref = []#获取参考碱基
                for i in list_jianji_qiepian:
                    list_jianji_ref.append(ref_seq_list[i])
            #     print(list_jianji_ref)

                #获取alt碱基
                list_jianji_alt= []#获取参考碱基
                for i in list_jianji_qiepian:
                    if i == int(gene_pos - 1):
                        list_jianji_alt.append(alt)
                    else:
                        list_jianji_alt.append(ref_seq_list[i])
            #     print(list_jianji_alt)
            #翻译蛋白质
            str_ref = "".join(list_jianji_ref)
            str_alt = "".join(list_jianji_alt)
            aa_ref = str(Seq(str_ref).translate())
            aa_alt = str(Seq(str_alt).translate())    
            #非同义：
            if aa_alt != aa_ref:
                info = f'{aa_ref}_{product}_{aa_pos}_{aa_alt}'
                dict_aa_mut[k] = {}
                dict_aa_mut[k]['product'] = dict_info_feikong[k]['product']
                dict_aa_mut[k]['aa'] = info
                dict_aa_mut[k]['num'] = dict_mut_num[k]
            #同义
            if aa_alt == aa_ref:
                info = f'{aa_ref}_{product}_{aa_pos}_{aa_alt}'
                dict_aa_synonymous_mut[k] = {}
                dict_aa_synonymous_mut[k]['product'] = dict_info_feikong[k]['product']
                dict_aa_synonymous_mut[k]['aa'] = info
                dict_aa_synonymous_mut[k]['num'] = dict_mut_num[k]
        #             dict_aa_mut[k]['frequency'] = dict_info_feikong[k]['frequency']

        # 

        #非同义----------------------------------------------------------------------
        #统计各个突变的频率。
        with open(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/08product_frequency/{c1}/08isnv_nonsynonymous_mut_frequency.csv','w') as f:
            writer = csv.writer(f)
            writer.writerow(['isnv','product','aa_mutation','sample_number'])
            for k,v in dict_aa_mut.items():
                writer.writerow([k,v['product'],v['aa'],v['num']])


        #统计非同义突变在哪个蛋白上nonsynonymous mutation
        list_product = []
        for k,v in dict_aa_mut.items():
                if v['product'] not in list_product:
                    list_product.append(v['product'])
                    dict_product = dict.fromkeys(list_product)#每个蛋白上非同义突变的个数
        for k in dict_product.keys():
            dict_product[k] = 0
        for m,p in dict_aa_mut.items():
            for k,v in dict_product.items():
                if p['product'] == k:
                    dict_product[k] += int(p['num']) #赋值的时候不可以直接写v，要写成k的值
        # print(dict_product)
        dict_product = dict(sorted(dict_product.items(),key=lambda x:x[0]))
        # print(dict_product)
        with open(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/08product_frequency/{c1}/08isnv_nonsynonymous_product_number.csv','w') as f:
            writer = csv.writer(f)
            writer.writerow(['product','number','group'])
            for k,v in dict_product.items():
                writer.writerow([k,v,'nonsynonymous'])


        #同义----------------------------------------------------------------------
        #统计各个突变的频率。
        with open(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/08product_frequency/{c1}/08isnv_synonymous_mut_frequency.csv','w') as f:
            writer = csv.writer(f)
            writer.writerow(['isnv','product','aa_mutation','sample_number'])
            for k,v in dict_aa_synonymous_mut.items():
                writer.writerow([k,v['product'],v['aa'],v['num']])



        #统计同义突变在哪个蛋白上
        list_product = []
        for k,v in dict_aa_synonymous_mut.items():
                if v['product'] not in list_product:
                    list_product.append(v['product'])
                    dict_product = dict.fromkeys(list_product)
        for k in dict_product.keys():
            dict_product[k] = 0
        for m,p in dict_aa_synonymous_mut.items():
            for k,v in dict_product.items():
                if p['product'] == k:
                    dict_product[k] += int(p['num']) #赋值的时候不可以直接写v，要写成k的值
        # print(dict_product)
        dict_product = dict(sorted(dict_product.items(),key=lambda x:x[0]))
        # print(dict_product)
        with open(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/08product_frequency/{c1}/08isnv_synonymous_product_number.csv','w') as f:
            writer = csv.writer(f)
            writer.writerow(['product','number','group'])
            for k,v in dict_product.items():
                writer.writerow([k,v,'synonymous'])

In [ ]:
df = pd.read_csv('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/06product_exist/06snp_mutation_PRJNA438443.csv')
df_counts = df.isnv.value_counts()
print(df_counts)
# outputpath=f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/06product_exist/mut_sample_number/{c1}_mut_sample_number.csv'
# df_counts.to_csv(outputpath,sep=',',index=True,header=False)



In [ ]:
#加入每个突变的样本数量.不分批次的
# 05 翻译蛋白质，要加3-80%的。isnv的
#非同义氨基酸突变的具体情况和不同蛋白非同义突变的数量。
#插入没加先。
import csv
import pandas as pd
from Bio.Seq import Seq
dict_info = {}
info_list = []
dict_pep_cds = {'99_518':'NS1_1C','628_1002':"NS2_1B",'1140_2315':"N",'2348_3073':"P",'3233_4003':"M",'4274_4468':"SH",'4659_5555':"G",'5632_7356':"F",'7576_8160':"M2",'8468_14965':"L"}
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/workflow_result_pair/06product_exist/06isnv_mutation.csv') as csvfile:
    csv_reader = csv.reader(csvfile)  
    for row in csv_reader:
        if row[0] != 'isnv':
            mut = row[0]
            if mut not in info_list:
                info_list.append(mut)
dict_info = dict.fromkeys(info_list)


for i in dict_info.keys():#嵌套字典可以这样赋值：1、dict_info = dict.fromkeys(info_list)2、每个键都赋值成空字典，再正常加就好了
    dict_info[i] = {}
#新加的，统计每个突变的样本数量。
df = pd.read_csv('/mnt/alamo01/users/yangjingqi/RSV_sra/group/workflow_result_pair/06product_exist/06isnv_mutation.csv')
df_counts = df.isnv.value_counts()
# print(df_counts)
outputpath=f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/workflow_result_pair/06product_exist/mut_sample_number.csv'
df_counts.to_csv(outputpath,sep=',',index=True,header=False)

dict_mut_num = {}

mut_number_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/workflow_result_pair/06product_exist/mut_sample_number.csv'
with open(mut_number_path) as csvfile:
    csv_reader = csv.reader(csvfile) 
    for row in csv_reader:
        isnv = row[0]
        num = row[1]
        dict_mut_num[isnv] = num    

    
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/workflow_result_pair/06product_exist/06isnv_mutation.csv') as csvfile:
    csv_reader = csv.reader(csvfile)  
    for row in csv_reader:
        if row[0] != 'isnv':
            mut = row[0]
            pos = int(row[0].split('_')[1])
            fre = float(row[2])
            type4 = row[3]
            first = row[0].split('_')[0]
            last = row[0].split('_')[2]
            #分一下位置，del和ins的有可能结束位置和开始位置不在同一段蛋白里面。不太可能
            if type4 == 'del':#按照突变前的长度算
                pos_end = pos + len(first) -1
            if type4 == 'ins':#按照突变后的长度算
                pos_end = pos + len(last) -1          
            for k,v in dict_pep_cds.items():
                start = int(k.split('_')[0])
                end = int(k.split('_')[1])
                if type4 == 'snp':
                    if start <= pos <= end: #这里一定要转化成int，不然就是字符串是不对的。    
                        dict_info[mut]['product'] = v
                        dict_info[mut]['type'] = type4
                        dict_info[mut]['start'] = start
                        dict_info[mut]['end'] = end
                        dict_info[mut]['sample_number'] = dict_mut_num[mut]
#                         dict_info[mut]['frequency'] = fre 频率先不考虑了，有好多样本的
                if type4 == 'del' or type4 == 'ins':
                    if start <= pos <= end and start <= pos_end <= end: #这里一定要转化成int，不然就是字符串是不对的。    
                        dict_info[mut]['product'] = v
                        dict_info[mut]['type'] = type4
                        dict_info[mut]['start'] = start
                        dict_info[mut]['end'] = end
                        dict_info[mut]['sample_number'] = dict_mut_num[mut]
#                         dict_info[mut]['frequency'] = fre

                                       
# print(dict_info)
#去除不在cds范围内的碱基突变                   
dict_info_feikong = {}            
for i in dict_info.keys():
    if dict_info[i]:
        dict_info_feikong[i] = dict_info[i]
# print(dict_info_feikong)

#获取突变前后的三个碱基，蛋白位置不知道，可以用什么蛋白
dict_aa_mut = {}#非同义突变的字典
dict_aa_synonymous_mut = {}
ref_seq = 'ACGCGAAAAAATGCGTACAACAAACTTGCGTAAACCAAAAAAATGGGGCAAATAAGAATTTGATAAGTACCACTTAAATTTAACTCCCTTGGTTAGAGATGGGCAGCAATTCATTGAGTATGATAAAAGTTAGATTACAAAATTTGTTTGACAATGATGAAGTAGCATTGTTAAAAATAACATGCTATACTGACAAATTAATACATTTAACTAATGCATTGGCTAAGGCAGTGATACATACAATCAAATTGAATGGCATTGTATTTGTGCATGTTATTACAAGTAGTGATATTTGCCCTAATAATAATATTGTAGTGAAATCCAATTTCACAACAATGCCAGTGTTACAAAATGGAGGTTATATATGGGAAATGATGGAATTAACACACTGCTCTCAACCTAATGGCCTAATAGATGACAATTGTGAAATTAAATTCTCCAAAAAACTAAGTGATTCAACAATGACCAATTATATGAATCAATTATCTGAATTACTTGGATTTGATCTTAATCCATAAATTATAATAAATATCAACTAGCAAATCAATGTCACTAACACCATTAGTTAATATAAAACTTGACAGAAGATAAAAATGGGGCAAATAAATCAATTCAGCCGACCCAACCATGGACACAACACACAATGACACCACACCACAAAGACTGATGATCACAGACATGAGACCATTGTCACTTGAGACTATAATAATATCACTAACCAGAGACATCATAACACACAGATTTATATACTTGATAAATCATGAATGTATAGTGAGAAAACTTGATGAAAGACAGGCCACATTTACATTCCTGGTCAACTATGAAATGAAACTATTGCACAAAGTGGGAAGCACTAAATACAAAAAATATACTGAATACAACACAAAATATGGCACTTTTCCTATGCCAATATTTATCAATCATGATGGGTTCTTAGAATGCATTGGCATTAAGCCTACAAAGCACACTCCCATAATATACAAGTATGATCTCAATCCATGAATTTCAACACAAGAGTCACACAATCTGAAATAACAACTTCATGCATAACCACACTCCATAGTTCAAATGGAGCCTGAAAATTATAGTAATTTAAAATTAAGGAGAGACATAAGATGAAAGATGGGGCAAATACAAAAATGGCTCTTAGCAAAGTCAAGTTGAATGATACACTCAACAAAGATCAACTTCTGTCATCCAGCAAATACACCATCCAACGGAGCACAGGAGATAGTATTGATACTCCTAATTATGATGTGCAGAAACACATCAATAAGTTATGTGGCATGTTATTAATCACAGAAGATGCTAATCATAAATTCACTGGGTTAATAGGTATGTTATATGCTATGTCTAGATTAGGAAGAGAAGACACCATAAAAATACTCAGAGATGCGGGATATCATGTAAAAGCAAATGGAGTGGATGTAACAACACATCGTCAAGATATTAATGGGAAAGAAATGAAATTTGAAGTGTTAACATTGTCAAGCTTAACAACTGAAATTCAAATCAACATTGAGATAGAATCTAGAAAATCCTACAAAAAAATGCTAAAAGAAATGGGAGAGGTAGCTCCAGAATACAGGCATGACTCTCCTGATTGTGGGATGATAATATTATGTATAGCGGCATTAGTAATAACCAAATTAGCAGCAGGGGATAGATCTGGTCTTACAGCTGTGATTAGGAGGGCTAATAATGTCCTAAAAAATGAAATGAAACGTTATAAAGGCTTACTACCCAAGGATATAGCCAACAGCTTCTATGAAGTGTTTGAAAAATATCCTCACTTTATAGATGTTTTTGTTCATTTTGGTATAGCACAATCTTCTACCAGAGGTGGCAGTAGAGTTGAAGGGATTTTTGCTGGATTGTTTATGAATGCCTATGGTGCAGGGCAAGTGATGTTACGGTGGGGGGTCTTAGCAAAATCAGTTAAAAATATTATGCTAGGACACGCTAGTGTGCAAGCAGAAATGGAACAAGTTGTGGAGGTTTATGAATATGCCCAAAAATTGGGTGGAGAAGCAGGGTTCTACCATATATTGAACAACCCAAAAGCATCATTATTGTCTTTGACTCAATTTCCTCACTTCTCCAGTGTAGTATTAGGCAATGCTGCTGGCCTAGGCATAATGGGAGAATACAGAGGTACACCAAGGAATCAAGATCTATATGATGCTGCAAAAGCATATGCTGAACAACTCAAAGAAAATGGTGTGATTAACTACAGTGTATTAGACTTGACAGCAGAAGAACTAGAGGCTATCAAACATCAGCTTAATCCAAAAGATAATGATGTAGAGCTTTGAGTTAATAAAAAAAATGGGGCAAATAAAACATCATGGAAAAGTTTGCTCCTGAATTCCATGGAGAAGATGCAAACAACAGAGCTACCAAATTCCTAGAATCAATAAAGGGCAAATTCACATCACCTAAAGATCCCAAGAAAAAAGATAGTATCATATCTGTCAACTCAATAGATATAGAAGTAACCAAAGAAAGCCCTATAACATCAAATTCAACCATTATAAACCCAACAAATGAGACAGATGATACTGTAGGGAACAAGCCCAATTATCAAAGAAAACCTCTAGTAAGTTTCAAAGAAGACCCTACGCCAAGTGATAATCCCTTTTCAAAACTATACAAAGAAACCATAGAAACATTTGATAACAATGAAGAAGAATCTAGCTATTCATATGAAGAAATAAATGATCAGACAAACGATAATATAACAGCAAGATTAGATAGGATTGATGAAAAATTAAGTGAAATACTAGGAATGCTTCACACATTAGTAGTAGCGAGTGCAGGACCTACATCTGCTCGGGATGGTATAAGAGATGCCATGGTTGGTTTAAGAGAAGACATGATAGAAAAAATCAGAACTGAAGCATTAATGACCAATGACAGACTAGAAGCTATGGCAAGACTCAGGAATGAGGAAAGTGAAAAGATGGCAAAAGACACATCAGATGAAGTGTCTCTCAATCCAACATCAGAGAAATTGAACAACCTGTTGGAAGGGAATGATAGTGACAATGATCTATCACTTGATGATTTCTGATCAGTTACCAATCTGTACATCAACACACAACACCAACAGAAGACCAACAAACAAACCAACTCACCCATCCAACCAAACATCTATACGCCAATCAGCCAATCCAAAACTAGCCACCCGGAAAAAATAGATACTATAGTTACAAAAAAAGATGGGGCAAATATGGAAACATACGTGAACAAACTTCACGAAGGCTCCACATACACAGCTGCTGTTCAATACAATGTCTTAGAAAAAGACGATGACCCTGCATCACTTACAATATGGGTGCCCATGTTCCAATCATCCATGCCAGCAGATTTACTTATAAAAGAACTAGCTAATGTCAACATACTAGTGAAACAAATATCCACACCCAATGGACCTTCATTAAGAGTCATGATAAACTCAAGAAGTGCAGTGCTAGCACAAATGCCCAGCAAATTTACCATATGTGCCAATGTGTCCTTGGATGAAAGAAGCAAGCTGGCATATGATGTAACCACACCCTGTGAAATCAAGGCATGTAGTCTAACATGCCTAAAATCAAAAAATATGTTAACTACAGTTAAAGATCTCACTATGAAAACACTCAACCCAACACATGACATCATTGCTTTATGTGAATTTGAAAATATAGTAACATCAAAAAAAGTCATAATACCAACATACCTAAGATCCATCAGTGTCAGAAATAAAGATCTGAACACACTTGAAAATATAACAACCACTGAATTCAAAAATGCCATCACAAATGCAAAAATCATCCCTTACTCAGGATTACTGTTAGTCATCACAGTGACTGACAACAAAGGAGCATTCAAATACATAAAGCCACAAAGTCAATTTATAGTAGATCTTGGAGCTTACCTAGAAAAAGAAAGTATATATTATGTTACAACAAATTGGAAGCACACAGCTACACGATTTGCAATCAAACCCATGGAAGATTAACCTTTTTCTTCTACATCAGTGAGTTGATTCATACAAACTTTCTACCTACATTCTTCACTTCACCATCATAATCACCAACCCTCTGTGGTTCAACTAATCAAACAAAACCCATCTGGAGCCTCAGATCATCCCAAGTCATTGTTCATCAGATCTAGTACTCAAATAAGTTAATAAAAATATCCACATGGGGCAAATAATCATTGGAGGAAATCCAACTAATCACAATATCTGTCAACATAGACAAGTCAACACGCCAGGCAAAATCAACCAATGGAAAATACATCCATAACAATAGAATTCTCAAGCAAATTCTGGCCTTACTTTACACTAATACACATGATAACAACAATAATCTCTTTGCTAATCATAATCTCCATCATGATTGCAATACTGAACAAACTCTGTGAATATAACGTATTCCATAACAAAACCTTTGAGCTACCAAGAGCTCGAGTCAATACATAGCATTCACCAATCTGATGGCACAAAACAGTAACCTTGCATTTGTAAGTGAACAACCCTCACCTCTTTACAAAACCACATCAACATCTCACCATGCAAGCCATCATCCATATTATAAAGTAGTTAATTAAAAATAATCATAACAATGAACTAAGATATTAAGACTAACAATAACGTTGGGGCAAATGCAAACATGTCCAAAAACAAGGACCAACGCACCACCAAGACACTAGAAAAGACCTGGGACACTCTCAATCATCTATTATTCATATCATCGTGCTTATACAAGTTAAATCTTAAATCTATAGCACAAATCACATTATCCATTCTGGCAATGATAATCTCAACTTCACTTATAATTGCAGCCATCATATTCATAGCCTCGGCAAACCACAAAGTCACACTAACAACTGCAATCATACAAGATGCAACAAGCCAGATCAAGAACACAACCCCAACATACCTCACCCAGAATCCCCAGCTTGGAATCAGCTTCTCCAATCTGTCTGAAACTACATCACAAACCACCACCATACTAGCTTCAACAACACCAAGTGTCAAGTCAACCCTGCAATCCACAACAGTCAAGACCAAAAACACAACAACAACCAAAATACAACCCAGCAAGCCCACCACAAAACAACGCCAAAACAAACCACCAAACAAACCCAATAATGATTTTCACTTTGAAGTGTTCAACTTTGTACCTTGCAGCATATGCAGCAACAATCCAACCTGCTGGGCTATCTGTAAAAGAATACCAAACAAAAAACCTGGAAAGAAAACCACCACCAAGCCCACAAAAAAACCAACCATCAAGACAACCAAAAAAGATCTCAAACCTCAAACCACAAAACCAAAGGAAGTACCTACCACCAAGCCCACAGAAAAGCCAACCATCAACACCACCAAAACAAACATCAGAACTACACTGCTCACCAACAATACCACAGGAAATCCAGAACACACAAGTCAAAAGGGAACCCTCCACTCAACCTCCTCCGATGGCAATCCAAGCCCTTCACAAGTCTATACAACATCCGAGTACCTATCACAACCTCCATCTCCATCCAACACAACAAACCAGTAGTCATTAAAAAGCGTATTATTGCAAAAAGCCATGACCAAATCAACCAGAATCAAAATCAACTCTGGGGCAAATAACAATGGAGTTGCCAATCCTCAAAACAAATGCAATTACCGCAATCCTTGCTGCAGTCACACTCTGTTTTGCTTCCAGTCAAAACATCACTGAAGAATTTTATCAAACAACATGCAGTGCAGTCAGCAAAGGCTATCTTAGTGCTCTAAGAACTGGTTGGTATACTAGTGTTATAACTATAGAATTAAGTAATATCAAGGAAAATAAGTGTAATGGAACAGACGCTAAGGTAAAATTGATAAAACAAGAATTAGATAAATATAAAAGTGCTGTAACAGAATTGCAGTTGCTCATGCAAAGCACACCGGCAACCAACAATCGAGCCAGAAGAGAACTACCAAGGTTTATGAATTATACACTCAACAATACCAAAAATACCAATGTAACATTAAGCAAGAAAAGGAAAAGAAGATTTCTTGGCTTTTTGTTAGGTGTTGGATCTGCAATCGCCAGTGGCATTGCTGTATCTAAGGTCCTGCACCTAGAAGGGGAAGTGAACAAAATCAAAAGTGCTCTACTATCCACAAACAAGGCTGTAGTCAGCTTATCAAATGGAGTTAGTGTCTTAACCAGCAAAGTGTTAGACCTCAAAAACTATATAGATAAACAGTTGTTACCTATTGTGAACAAGCAAAGCTGTAGCATATCAAACATTGAAACTGTGATAGAGTTCCAACAAAAGAACAACAGACTACTAGAGATTACCAGGGAATTTAGTGTTAATGCAGGTGTAACTACACCTGTAAGCACTTATATGTTAACAAATAGTGAATTATTATCATTAATCAATGATATGCCTATAACAAATGATCAGAAAAAGTTAATGTCCAACAATGTTCAAATAGTTAGACAGCAAAGTTACTCTATCATGTCCATAATAAAGGAGGAAGTCTTAGCATATGTAGTACAATTACCACTATATGGTGTAATAGATACACCTTGTTGGAAACTGCACACATCCCCTCTATGTACAACCAACACAAAGGAAGGGTCCAACATCTGTTTAACAAGAACCGACAGAGGATGGTACTGTGACAATGCAGGATCAGTATCTTTCTTCCCACTAGCTGAAACATGTAAAGTTCAATCGAATCGAGTATTTTGTGACACAATGAACAGTTTAACATTACCAAGTGAAGTAAATCTCTGCAACATTGACATATTCAACCCCAAATATGATTGCAAAATTATGACTTCAAAAACAGATGTAAGCAGCTCCGTTATCACATCTCTAGGAGCCATTGTGTCATGCTATGGCAAAACTAAATGTACAGCATCCAATAAAAATCGTGGAATCATAAAGACATTTTCTAACGGGTGCGATTATGTATCAAATAAGGGGGTTGACACTGTGTCTGTAGGTAATACATTATATTATGTAAATAAGCAAGAAGGCAAAAGTCTCTATGTAAAAGGTGAACCAATAATAAATTTCTATGACCCATTAGTGTTCCCCTCTGATGAATTTGATGCATCAATATCTCAAGTCAATGAGAAGATTAACCAGAGCCTAGCATTTATTCGTAAATCCGATGAATTATTACATAATGTAAATGCTGGTAAATCCACCATAAATATCATGATAACTACTATAATTATAGTGATTATAGTAATATTGTTATCATTAATTGCCGTTGGACTGCTCCTATACTGCAAGGCCAGAAGCACACCAGTCACACTAAGCAAGGATCAACTGAGTGGTATAAATAATATTGCATTTAGTAACTAAATAAAAATAGCACCTAATCATGTTCTTACAATGGTTTCATATCTGCTCATAGACAACCCATCTATCATTGGATTTTCTTAAAATCTGAACTTCATCGAAACTCTCATCTATAAACCATCTCACTTACATTATTTAAGTAGATTCCTAGTTTATAGTTATATAAAACAATTGAATACCAGATTAACTTACTATTTGTAAAAAATGAGAACTGGGGCAAATATGTCACGAAGGAATCCTTGCAAATTTGAAATTCGAGGTCATTGCTTGAATGGTAAGAGGTGTCATTTTAGTCATAATTATTTTGAATGGCCACCCCATGCACTGCTTGTAAGACAAAACTTTATGTTAAACAGAATACTTAAGTCTATGGATAAAAGCATCGATACTTTATCAGAAATAAGTGGAGCTGCAGAGTTGGACAGAACAGAAGAGTATGCCCTCGGTGTAGTTGGAGTGCTAGAGAGTTATATAGGATCTATAAATAATATAACTAAACAATCAGCATGTGTTGCCATGAGCAAACTCCTCACTGAACTCAACAGTGATGACATCAAAAAACTGAGGGACAATGAAGAGCCAAATTCACCCAAGATAAGAGTGTACAATACTGTCATATCATATATTGAAAGCAACAGGAAAAACAATAAACAAACTATCCATCTGTTAAAAAGATTGCCAGCAGACGTATTGAAGAAAACCATAAAAACCACATTGGATATCCACAAGAGCATAACCATCAATAACCCAAAAGAATCAACTGTTAGTGATATAAACGACCATGCCAAAAATAATGATACTACCTGACAAATATCCTTGTAGTATAAATTCCATACTAATAACAAGTAGTTGTAGAGTTACTATGTATAATCAAAAGAACACACTATATTTCAATCAAAACAACCAAAATAACCATATATACTCACCGAATCAACCATTCAATGAAATCCATTGGACCTCTCAAGACTTGATTGATGCAATTCAAAATTTTCTACAACATCTAGGTATTACTGATGATATATACACAATATATATATTAGTGTCATAACACTCAATCCTAATGCTTACCACATCATCAAACTATTAACTCAAACAATTCAAGCCATGGGACAAAATGGATCCCATTATTAATGGAAATTCTGCTAATGTGTATCTAACCGATAGTTATTTAAAAGGTGTTATTTCTTTCTCAGAATGTAATGCTTTAGGAAGTTACATATTCAATGGTCCTTATCTCAAAAATGATTATACCAACTTAATTAGTAGACAAAATCCATTAATAGAACACATAAATCTAAAGAAACTAAATATAACACAGTCCTTAATGTCTAAGTATCATAAAGGTGAAATAAAAATAGAAGAACCTACTTATTTTCAGTCATTACTTATGACATACAAGAGTATGACCTCGTTAGAACAGATTACTACCACTAATTTACTTAAAAAGATAATAAGAAGAGCTATAGAAATTAGTGATGTCAAAGTCTATGCTATATTGAATAAACTGGGGCTTAAAGAAAAAGACAAGATTAAATCCAACAATGGACAAGATGAAGACAACTCAGTTATTACAACCATAATCAAAGATGATATACTTTTAGCTGTTAAGGATAATCAATCTCATCTTAAAGCAGTCAAAAATCACTCTACAAAACAAAAAGATACAATCAAAACAACACTCTTGAAGAAATTAATGTGTTCAATGCAACATCCTCCATCATGGTTAATACATTGGTTTAATTTATACACAAAATTAAACAACATATTAACACAGTATCGATCAAGTGAGGTAAAAAACCATGGTTTTATATTGATAGACAATCATACTCTCAATGGATTCCAATTTATTTTGAATCAATATGGTTGTATAGTTTATCATAAGGAACTCAAAAGAATTACTGTGACAACCTATAATCAATTCTTGACATGGAAAAATATTAGCCTTAGTAGATTAAATGTTTGTTTAATTACATGGATTAGTAACTGTTTGAACACATTAAATAAAAGCTTAGGCTTAAGATGCGGATTCAATAATGTTATCTTGACACAACTATTCCTCTATGGAGATTGTATACTAAAACTATTCCACAATGAGGGGTTCTACATAATAAAAGAGGTAGAGGGATTTATTATGTCTCTAATTTTAAATATAACAGAAGAAGATCAATTCAGAAAACGGTTTTATAATAGTATGCTCAACAACATCACAGATGCTGCTAATAAAGCTCAGAAAAATCTGCTATCAAGAGTATGTCATACATTATTAGATAAGACAGTATCCGATAATATAATAAATGGCAGATGGATAATTCTATTAAGTAAGTTCCTTAAATTAATTAAGCTTGCAGGTGACAATAACCTTAACAATCTGAGTGAATTATATTTTTTGTTCAGAATATTTGGACACCCAATGGTAGATGAAAGACAAGCCATGGATGCTGTTAAAGTTAATTGCAACGAGACCAAATTTTACTTGTTAAGCAGTTTGAGTATGTTAAGAGGTGCCTTTATATATAGAATTATAAAAGGATTTGTAAATAATTACAACAGATGGCCTACTTTAAGGAATGCTATTGTTTTACCCTTAAGATGGTTAACTTACTATAAACTAAACACTTATCCTTCCTTGTTGGAACTTACAGAAAGAGATTTGATTGTTTTATCAGGACTACGTTTCTATCGTGAGTTTCGGTTGCCTAAAAAAGTGGATCTTGAAATGATCATAAATGATAAGGCTATATCACCTCCTAAAAATTTGATATGGACTAGTTTCCCTAGAAATTATATGCCGTCACACATACAAAATTATATAGAACATGAAAAATTAAAATTTTCCGAGAGTGATAAATCAAGAAGAGTATTAGAGTACTATTTAAGAGATAACAAATTCAATGAATGTGATTTATACAACTGTGTAGTTAATCAAAGTTATCTTAACAACCCTAATCATGTGGTATCTTTGACAGGCAAAGAAAGAGAACTCAGTGTAGGTAGAATGTTTGCAATGCAACCAGGAATGTTCAGACAAGTTCAAATATTAGCAGAGAAAATGATAGCTGAAAACATTTTACAATTCTTTCCTGAAAGTCTTACAAGATATGGTGATCTAGAACTACAGAAAATATTAGAATTGAAAGCAGGAATAAGTAACAAATCAAATCGTTACAATGATAATTACAACAATTACATTAGTAAGTGCTCTATCATCACAGATCTCAGCAAATTCAATCAAGCATTTCGATATGAAACATCATGTATTTGTAGTGATGTACTGGATGAACTGCATGGTGTACAATCTCTATTTTTCTGGTTACATTTAGCTATTCCTCATGTCACAATAATATGCACATATAGGCATGCACCCCCCTATATAAGAGATCATATTGTAGATCTTAACAATGTAGATGAACAAAGTGGATTATATAGATATCATATGGGTGGTATCGAAGGGTGGTGTCAAAAACTATGGACCATAGAAGCTATATCACTATTGGATCTAATATCTCTCAAAGGGAAATTCTCAATTACTGCTTTAATTAATGGTGACAATCAATCAATAGATATAAGTAAACCAGTCAGACTCATGGAAGGTCAAACTCATGCTCAAGCAGATTATTTGCTAGCATTAAATAGTCTTAAATTACTGTATAAAGAGTATGCAGGCATAGGCCACAAATTAAAAGGAACTGAGACTTATATATCAAGAGATATGCAATTTATGAGTAAAACAATTCAACATAACGGTGTATATTACCCAGCTAGTATAAAGAAAGTCCTAAGAGTGGGACCGTGGATAAACACTATACTTGATGATTTCAAAGTGAGTCTAGAATCTATAGGTAGTTTGACACAAGAATTAGAATATAGAGGAGAAAGTCTATTATGCAGTTTAATATTTAGAAATGTATGGTTATATAATCAAATTGCTTTACAACTAAAAAATCATGCATTATGTAACAATAAATTATATTTGGACATATTAAAGGTTCTGAAACACTTAAAAACCTTTTTTAATCTTGATAATATTGATACAGCATTAACATTGTATATGAATTTGCCCATGTTATTTGGTGGTGGTGATCCCAACTTGTTATATCGAAGTTTCTATAGAAGAACTCCTGATTTCCTCACAGAGGCTATAGTTCACTCTGTGTTCATACTTAGTTATTATACAAACCATGATTTAAAAGATAAACTTCAAGATCTGTCAGATGATAGATTGAATAAGTTCTTAACATGCATAATCACATTTGACAAAAACCCTAATGCTGAATTCGTAACATTGATGAGAGATCCTCAAGCTTTAGGGTCTGAGAGACAAGCTAAAATTACTAGCGAAATCAATAGACTGGCAGTTACTGAGGTTTTGAGCACAGCTCCAAACAAAATATTCTCCAAAAGTGCACAACACTATACCACTACAGAGATAGATCTAAATGATATTATGCAAAATATAGAACCTACATATCCTCACGGGCTAAGAGTTGTTTATGAAAGTTTACCCTTTTATAAAGCAGAGAAAATAGTAAATCTTATATCCGGTACAAAATCTATAACTAACATACTGGAAAAGACTTCTGCCATAGACTTAACAGATATTGATAGAGCCACTGAGATGATGAGGAAAAACATAACTTTGCTTATAAGGATATTTCCATTAGATTGTAACAGAGATAAAAGGGAAATATTGAGTATGGAAAACCTAAGTATTACTGAATTAAGCAAATATGTTAGGGAAAGATCTTGGTCTTTATCCAATATAGTTGGTGTTACATCACCTAGTATCATGTATACAATGGACATCAAATATACAACAAGCACTATAGCTAGTGGCATAATCATAGAGAAATATAATGTTAACAGTTTAACACGTGGTGAGAGAGGACCCACTAAACCATGGGTTGGTTCATCTACACAAGAGAAAAAAACAATGCCAGTTTATAATAGACAAGTTTTAACCAAAAAACAGAGAGATCAAATAGATCTATTAGCAAAATTGGATTGGGTGTATGCATCTATAGATAACAAGGATGAATTCATGGAAGAACTTAGCATAGGAATTCTTGGGTTAACATATGAGAAAGCCAAAAAATTATTTCCACAATATTTAAGTGTTAACTATTTGCATCGCCTTACAGTCAGTAGTAGACCATGTGAATTCCCTGCATCAATACCAGCTTATAGAACTACAAATTATCACTTTGATACTAGCCCTATTAATCGCATATTAACAGAAAAGTATGGTGATGAAGATATTGATATAGTATTCCAAAACTGTATAAGCTTTGGCCTTAGCTTAATGTCAGTAGTAGAACAATTTACTAATGTATGTCCTAACAGAATTATTCTTATACCTAAGCTTAATGAGATACATTTAATGAAACCTCCCATATTCACAGGTGATGTTGATATTCACAAGTTAAAACAAGTGATACAAAAACAGCATATGTTTTTACCAGACAAAATAAGTTTGACTCAATATGTGGAATTATTCTTAAGTAATAAAACACTCAAATCTGGATCTCATGTTAATTCTAATTTAATATTGGCGCATAAGATATCTGACTATTTTCATAATACTTACATTTTAAGTACTAATTTAGCTGGACATTGGATTCTGATTATACAACTTATGAAAGATTCTAAGGGTATTTTTGAAAAAGATTGGGGAGAGGGATATATAACTGATCATATGTTCATTAATTTGAAAGTTTTCTTCAATGCTTATAAGACCTATCTCTTGTGTTTTCATAAAGGTTACGGCAGAGCAAAGCTGGAGTGTGATATGAATACTTCAGATCTCCTATGTGTATTGGAATTAATAGACAGTAGTTATTGGAAGTCTATGTCTAAGGTATTTTTAGAACAAAAAGTTATCAAATACATTCTTAGCCAGGATGCAAGTTTACATAGAGTAAAAGGATGTCATAGCTTCAAACTATGGTTTCTTAAACGTCTTAATGTAGCAGAATTCACAGTTTGCCCTTGGGTTGTTAACATAGATTATCATCCAACACATATGAAAGCAATATTAACTTATATAGATCTTGTTAGAATGGGATTGATAAATATAGATAAAATATACATTAAAAATAAACACAAATTCAATGATGAATTTTATACTTCTAATCTCTTTTACATTAATTATAACTTCTCAGATAATACTCATCTATTAACTAAACATATAAGGATTGCTAATTCTGAATTAGAAAATAATTACAACAAATTATATCATCCTACACCAGAAACCCTAGAAAATATACTAACCAATCCGGTTAAATGTAATGACAAAAAGACACTGAATGACTATTGTATAGGTAAAAATGTTGACTCAATAATGTTACCATTGTTATCTAATAAGAAGCTTATTAAATCGCCTACAATGATTAGAACCAATTACAGCAAACAAGATTTGTATAATTTATTTCCTACGGTTGTGATTGATAAAATTATAGATCATTCAGGTAATACAGCCAAATCTAACCAACTTTACACTACTACTTCTCATCAAATACCTTTAGTGCACAATAGCACATCACTTTATTGCATGCTTCCTTGGCATCATATTAATAGATTCAATTTTGTATTTAGTTCTACAGGTTGTAAAATTAGTATAGAGTATATTTTAAAAGACCTTATAATTAAAGATCCTAATTGTATAGCATTCATAGGTGAAGGAGCAGGGAATTTATTATTGCGTACAGTAGTGGAACTTCATCCCGATATAAGATATATTTACAGAAGTCTGAAGGATTGCAATGATCATAGTTTACCTATTGAGTTTTTAAGGCTGTACAATGGACATATCAACATTGATTATGGTGAAAATTTGACCATTCCTGCTACAGATGCAACCAACAACATTCATTGGTCTTATTTACATATAAAGTTTGCTGAACCTATCAGTCTTTTTGTCTGTGATGCTGAATTGCCTGTAACAGTCAACTGGAGTAAAATTATAATAGAGTGGAGCAAGCATGTAAGAAAATGCAAGTACTGTTCCTCAGTTAATAAATGTACGTTAATAGTAAAATATCATGCTCAAGATGATATCGATTTCAAATTAGACAATATAACTATATTAAAAACTTATGTATGCTTAGGCAGTAAGTTAAAGGGGTCTGAAGTTTACTTAGTCCTTACAATAGGTCCTGCAAATGTGTTCCCAGTATTTAATGTAGTACAAAATGCTAAATTGATACTATCAAGAACCAAAAATTTCATCATGCCTAAGAAGGCTGATAAAGAGTCTATTGATGCAAATATTAAAAGTTTGATACCCTTTCTTTGTTACCCTATAACAAAAAAAGGAATTAATACTGCATTGTCAAAACTAAAGAGTGTTGTTAGTGGAGATATACTATCATATTCTATAGCAGGACGTAATGAAGTTTTCAGCAATAAACTTATAAATCATAAGCATATGAACATCTTAAAATGGTTCAATCATGTTTTAAATTTCAGATCAACAGAACTAAACTATAATCATTTATATATGGTAGAATCTACATATCCTTATCTAAGTGAATTGTTAAACAGCTTGACAACTAATGAACTTAAAAAACTGATTAAAATCACAGGTAGTTTGTTATACAACTTTCATAATGAATAATGAATAAAAATCTTATATTAAAAATTCCCATAGCTACACACTAACACTGTATTCAATTATAGTTATTTAAAATTAAAAATTATATAATTTTTTAATAACTTTTAGTGAACTAATCCTAAAATTATCATTTTGATCTAGGAGGAATAAATTTAAATCCAAATCTAATTGGTTTATATGTATATTAACTAAACTACGAGATATTAGTTTTTGACACTTTTTTTCTCGT'
ref_seq_list = list(ref_seq)
dict_jianji = {}
for k,v in dict_info_feikong.items():
    gene_pos = int(k.split('_')[1])
    ref = k.split('_')[0]
    alt = k.split('_')[2]
    product = v['product']
#     frequency = v['frequency']
    gene_pos_f2 = gene_pos - 2   #前面2个
    gene_pos_f1 = gene_pos - 1   #前面1个
    gene_pos_b1 = gene_pos + 1
    gene_pos_b2 = gene_pos + 2
    start = v['start']
    end = v['end']
    type4 = v['type']
    chazhi = int(gene_pos) - int(start)
    quzheng =  chazhi//3   #取整，以获取是这个蛋白质的第几个氨基酸。
    aa_pos = quzheng + 1

# #分类讨论
    if type4 == 'del' or type4 == 'ins':
        del_count = len(ref) - len(alt)
        if del_count%3 == 0:
            #分类讨论
            #1
            if chazhi%3 == 2 and len(alt)%3 == 2:
                #变异前的位置和碱基
                list_jianji_pos_ref = []#是位置不是切片，切片要-1
                list_jianji_ref = []
                list_jianji_pos_ref.append(gene_pos_f2)
                list_jianji_pos_ref.append(gene_pos_f1)
                for index in range(0,len(ref)):
                    list_jianji_pos_ref.append((gene_pos + index))
                list_jianji_pos_ref.append(gene_pos_b1)
                list_jianji_pos_ref.append(gene_pos_b2)
                for p in list_jianji_pos_ref:
                    p_qiepian = p-1
                    list_jianji_ref.append(ref_seq[p_qiepian])
                    
                
                #变异后的位置
                list_jianji_pos_alt = []#是位置不是切片，切片要-1
                list_jianji_alt = []
                list_jianji_pos_alt.append(gene_pos_f2)
                list_jianji_pos_alt.append(gene_pos_f1)
                
                list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                list_jianji_alt.append(ref_seq[gene_pos_f1-1])
                for index in range(0,len(alt)):
                    list_jianji_pos_alt.append((gene_pos + index))
                    list_jianji_alt.append(alt[index])
                list_jianji_pos_alt.append(gene_pos_b1)
                list_jianji_pos_alt.append(gene_pos_b2)
                list_jianji_alt.append(ref_seq[gene_pos_b1-1])
                list_jianji_alt.append(ref_seq[gene_pos_b2-1])
           #2
            if chazhi%3 == 2 and len(alt)%3 == 1:
                list_jianji_pos_ref = []#是位置不是切片，切片要-1
                list_jianji_ref = []
                list_jianji_pos_ref.append(gene_pos_f2)
                list_jianji_pos_ref.append(gene_pos_f1)
                for index in range(0,len(ref)):
                    list_jianji_pos_ref.append((gene_pos + index))
              
                for p in list_jianji_pos_ref:
                    p_qiepian = p-1
                    list_jianji_ref.append(ref_seq[p_qiepian])
                    
                
                #变异后的位置
                list_jianji_pos_alt = []#是位置不是切片，切片要-1
                list_jianji_alt = []
                list_jianji_pos_alt.append(gene_pos_f2)
                list_jianji_pos_alt.append(gene_pos_f1)
                
                list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                list_jianji_alt.append(ref_seq[gene_pos_f1-1])
                for index in range(0,len(alt)):
                    list_jianji_pos_alt.append((gene_pos + index))
                    list_jianji_alt.append(alt[index])
           
            #3
            if chazhi%3 == 2 and len(alt)%3 == 0:
            #变异前的位置和碱基
                list_jianji_pos_ref = []#是位置不是切片，切片要-1
                list_jianji_ref = []
                list_jianji_pos_ref.append(gene_pos_f2)
                list_jianji_pos_ref.append(gene_pos_f1)
                for index in range(0,len(ref)):
                    list_jianji_pos_ref.append((gene_pos + index))
                list_jianji_pos_ref.append(gene_pos_b1)

                for p in list_jianji_pos_ref:
                    p_qiepian = p-1
                    list_jianji_ref.append(ref_seq[p_qiepian])


                #变异后的位置
                list_jianji_pos_alt = []#是位置不是切片，切片要-1
                list_jianji_alt = []
                list_jianji_pos_alt.append(gene_pos_f2)
                list_jianji_pos_alt.append(gene_pos_f1)

                list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                list_jianji_alt.append(ref_seq[gene_pos_f1-1])
                for index in range(0,len(alt)):
                    list_jianji_pos_alt.append((gene_pos + index))
                    list_jianji_alt.append(alt[index])
                list_jianji_pos_alt.append(gene_pos_b1)
                list_jianji_alt.append(ref_seq[gene_pos_b1-1])
            
            #4
            if chazhi%3 == 1 and len(alt)%3 == 2:
                #变异前的位置和碱基
                list_jianji_pos_ref = []#是位置不是切片，切片要-1
                list_jianji_ref = []
                list_jianji_pos_ref.append(gene_pos_f2)
               
                for index in range(0,len(ref)):
                    list_jianji_pos_ref.append((gene_pos + index))
              
                for p in list_jianji_pos_ref:
                    p_qiepian = p-1
                    list_jianji_ref.append(ref_seq[p_qiepian])
                    
                
                #变异后的位置
                list_jianji_pos_alt = []#是位置不是切片，切片要-1
                list_jianji_alt = []
                list_jianji_pos_alt.append(gene_pos_f2)
               
                
                list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                
                for index in range(0,len(alt)):
                    list_jianji_pos_alt.append((gene_pos + index))
                    list_jianji_alt.append(alt[index])
      
            #5
            if chazhi%3 == 1 and len(alt)%3 == 1:
                #变异前的位置和碱基
                list_jianji_pos_ref = []#是位置不是切片，切片要-1
                list_jianji_ref = []
                list_jianji_pos_ref.append(gene_pos_f2)
                for index in range(0,len(ref)):
                    list_jianji_pos_ref.append((gene_pos + index))
                list_jianji_pos_ref.append(gene_pos_b1)
                for p in list_jianji_pos_ref:
                    p_qiepian = p-1
                    list_jianji_ref.append(ref_seq[p_qiepian])
                    
                
                #变异后的位置
                list_jianji_pos_alt = []#是位置不是切片，切片要-1
                list_jianji_alt = []
                list_jianji_pos_alt.append(gene_pos_f2)
                list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                for index in range(0,len(alt)):
                    list_jianji_pos_alt.append((gene_pos + index))
                    list_jianji_alt.append(alt[index])
                list_jianji_pos_alt.append(gene_pos_b1)
                list_jianji_alt.append(ref_seq[gene_pos_b1-1])
        #6
            if chazhi%3 == 1 and len(alt)%3 == 0:
                #变异前的位置和碱基
                list_jianji_pos_ref = []#是位置不是切片，切片要-1
                list_jianji_ref = []
                list_jianji_pos_ref.append(gene_pos_f2)
                
                for index in range(0,len(ref)):
                    list_jianji_pos_ref.append((gene_pos + index))
                list_jianji_pos_ref.append(gene_pos_b1)
                list_jianji_pos_ref.append(gene_pos_b2)
                for p in list_jianji_pos_ref:
                    p_qiepian = p-1
                    list_jianji_ref.append(ref_seq[p_qiepian])
                    
                
                #变异后的位置
                list_jianji_pos_alt = []#是位置不是切片，切片要-1
                list_jianji_alt = []
                list_jianji_pos_alt.append(gene_pos_f2)
              
                
                list_jianji_alt.append(ref_seq[gene_pos_f2-1])
                
                for index in range(0,len(alt)):
                    list_jianji_pos_alt.append((gene_pos + index))
                    list_jianji_alt.append(alt[index])
                list_jianji_pos_alt.append(gene_pos_b1)
                list_jianji_pos_alt.append(gene_pos_b2)
                list_jianji_alt.append(ref_seq[gene_pos_b1-1])
                list_jianji_alt.append(ref_seq[gene_pos_b2-1])
                
            #7
            if chazhi%3 == 0 and len(alt)%3 == 2:
                #变异前的位置和碱基
                list_jianji_pos_ref = []#是位置不是切片，切片要-1
                list_jianji_ref = []
       
                for index in range(0,len(ref)):
                    list_jianji_pos_ref.append((gene_pos + index))
                list_jianji_pos_ref.append(gene_pos_b1)
               
                for p in list_jianji_pos_ref:
                    p_qiepian = p-1
                    list_jianji_ref.append(ref_seq[p_qiepian]) 
                #变异后的位置
                list_jianji_pos_alt = []#是位置不是切片，切片要-1
                list_jianji_alt = []
                for index in range(0,len(alt)):
                    list_jianji_pos_alt.append((gene_pos + index))
                    list_jianji_alt.append(alt[index])
                list_jianji_pos_alt.append(gene_pos_b1)
                list_jianji_alt.append(ref_seq[gene_pos_b1-1])
                
            #8
            if chazhi%3 == 0 and len(alt)%3 == 1:
                #变异前的位置和碱基
                list_jianji_pos_ref = []#是位置不是切片，切片要-1
                list_jianji_ref = []
                for index in range(0,len(ref)):
                    list_jianji_pos_ref.append((gene_pos + index))
                list_jianji_pos_ref.append(gene_pos_b1)
                list_jianji_pos_ref.append(gene_pos_b2)
                for p in list_jianji_pos_ref:
                    p_qiepian = p-1
                    list_jianji_ref.append(ref_seq[p_qiepian])
                    
                
                #变异后的位置
                list_jianji_pos_alt = []#是位置不是切片，切片要-1
                list_jianji_alt = []
                for index in range(0,len(alt)):
                    list_jianji_pos_alt.append((gene_pos + index))
                    list_jianji_alt.append(alt[index])
                list_jianji_pos_alt.append(gene_pos_b1)
                list_jianji_pos_alt.append(gene_pos_b2)
                list_jianji_alt.append(ref_seq[gene_pos_b1-1])
                list_jianji_alt.append(ref_seq[gene_pos_b2-1])
        #9
            if chazhi%3 == 0 and len(alt)%3 == 0:
                #变异前的位置和碱基
                list_jianji_pos_ref = []#是位置不是切片，切片要-1
                list_jianji_ref = []
                for index in range(0,len(ref)):
                    list_jianji_pos_ref.append((gene_pos + index))
         
                for p in list_jianji_pos_ref:
                    p_qiepian = p-1
                    list_jianji_ref.append(ref_seq[p_qiepian])
               
                #变异后的位置
                list_jianji_pos_alt = []#是位置不是切片，切片要-1
                list_jianji_alt = []
                for index in range(0,len(alt)):
                    list_jianji_pos_alt.append((gene_pos + index))
                    list_jianji_alt.append(alt[index])
      
 
    
  #------------------------原来的，底下可以不变  
    if type4 == 'snp':
        if chazhi%3 == 2:
            list_jianji_pos = []#是位置不是切片，切片要-1
            list_jianji_pos.append(gene_pos_f2)
            list_jianji_pos.append(gene_pos_f1)
            list_jianji_pos.append(gene_pos)



        if chazhi%3 == 1:
            list_jianji_pos = []
            list_jianji_pos.append(gene_pos_f1)
            list_jianji_pos.append(gene_pos)
            list_jianji_pos.append(gene_pos_b1)



        if chazhi%3 == 0:
            list_jianji_pos = []
            list_jianji_pos.append(gene_pos)
            list_jianji_pos.append(gene_pos_b1)
            list_jianji_pos.append(gene_pos_b2)


        list_jianji_qiepian = []#这个是切片
        for i in list_jianji_pos:
            i_1 = i-1
            list_jianji_qiepian.append(i_1)
    #     print(list_jianji_qiepian)

        list_jianji_ref = []#获取参考碱基
        for i in list_jianji_qiepian:
            list_jianji_ref.append(ref_seq_list[i])
    #     print(list_jianji_ref)

        #获取alt碱基
        list_jianji_alt= []#获取参考碱基
        for i in list_jianji_qiepian:
            if i == int(gene_pos - 1):
                list_jianji_alt.append(alt)
            else:
                list_jianji_alt.append(ref_seq_list[i])
    #     print(list_jianji_alt)
    #翻译蛋白质
    str_ref = "".join(list_jianji_ref)
    str_alt = "".join(list_jianji_alt)
    aa_ref = str(Seq(str_ref).translate())
    aa_alt = str(Seq(str_alt).translate())    
    #非同义：
    if aa_alt != aa_ref:
        info = f'{aa_ref}_{product}_{aa_pos}_{aa_alt}'
        dict_aa_mut[k] = {}
        dict_aa_mut[k]['product'] = dict_info_feikong[k]['product']
        dict_aa_mut[k]['aa'] = info
        dict_aa_mut[k]['num'] = dict_mut_num[k]
    #同义
    if aa_alt == aa_ref:
        info = f'{aa_ref}_{product}_{aa_pos}_{aa_alt}'
        dict_aa_synonymous_mut[k] = {}
        dict_aa_synonymous_mut[k]['product'] = dict_info_feikong[k]['product']
        dict_aa_synonymous_mut[k]['aa'] = info
        dict_aa_synonymous_mut[k]['num'] = dict_mut_num[k]
#             dict_aa_mut[k]['frequency'] = dict_info_feikong[k]['frequency']
    
# 

#非同义----------------------------------------------------------------------
#统计各个突变的频率。
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/workflow_result_pair/08product_frequency/08isnv_nonsynonymous_mut_frequency.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(['isnv','product','aa_mutation','sample_number'])
    for k,v in dict_aa_mut.items():
        writer.writerow([k,v['product'],v['aa'],v['num']])
    
    
#统计非同义突变在哪个蛋白上nonsynonymous mutation
list_product = []
for k,v in dict_aa_mut.items():
        if v['product'] not in list_product:
            list_product.append(v['product'])
            dict_product = dict.fromkeys(list_product)#每个蛋白上非同义突变的个数
for k in dict_product.keys():
    dict_product[k] = 0
for m,p in dict_aa_mut.items():
    for k,v in dict_product.items():
        if p['product'] == k:
            dict_product[k] += int(p['num']) #赋值的时候不可以直接写v，要写成k的值
# print(dict_product)
dict_product = dict(sorted(dict_product.items(),key=lambda x:x[0]))
# print(dict_product)
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/workflow_result_pair/08product_frequency/08isnv_nonsynonymous_product_number.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(['product','number','group'])
    for k,v in dict_product.items():
        writer.writerow([k,v,'nonsynonymous'])
        

#同义----------------------------------------------------------------------
#统计各个突变的频率。
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/workflow_result_pair/08product_frequency/08isnv_synonymous_mut_frequency.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(['isnv','product','aa_mutation','sample_number'])
    
    for k,v in dict_aa_synonymous_mut.items():
        writer.writerow([k,v['product'],v['aa'],v['num']])
    
    
#统计同义突变在哪个蛋白上
list_product = []
for k,v in dict_aa_synonymous_mut.items():
        if v['product'] not in list_product:
            list_product.append(v['product'])
            dict_product = dict.fromkeys(list_product)
for k in dict_product.keys():
    dict_product[k] = 0
for m,p in dict_aa_synonymous_mut.items():
    for k,v in dict_product.items():
        if p['product'] == k:
            dict_product[k] += int(p['num']) #赋值的时候不可以直接写v，要写成k的值
# print(dict_product)
dict_product = dict(sorted(dict_product.items(),key=lambda x:x[0]))
# print(dict_product)
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/workflow_result_pair/08product_frequency/08isnv_synonymous_product_number.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(['product','number','group'])
    for k,v in dict_product.items():
        writer.writerow([k,v,'synonymous'])

In [ ]:
#看一下都有哪些prj的vcF是没有找出来突变的
import os
def is_vcf_empty(vcf_file):
    with open(vcf_file, 'r') as f:
        for line in f:
            if not line.startswith('#'):
                return False
    return True

empty_list = []
prj_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/vcf_pair')
for i in prj_list:
    panduan_list = []
    vcf_list = os.listdir(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/vcf_pair/{i}')
    for n in vcf_list:
        vcf_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/vcf_pair/{i}/{n}'
# 调用函数来检查 VCF 文件是否为空
        vcf_empty = is_vcf_empty(vcf_path)
        if vcf_empty:
            panduan_list.append('empty')
        else:
            panduan_list.append('not_empty')
    if 'not_empty' in panduan_list:
        pass
    else:
        empty_list.append(i)
print(empty_list)







In [ ]:
#先找出来有哪些prj是有符合要求的突变，可以用的
import os
import csv


all_prj_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/fastq_result')
csv_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/09snp_isnv_all_mut')
not_empty_prj_list = []
for i in csv_list:
    csv_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/09snp_isnv_all_mut/{i}'
    total = len(open(csv_path).readlines())
    if total != 1:
        not_empty_prj_list.append(i.split('.')[0])
print(len(not_empty_prj_list))
print(not_empty_prj_list)

In [ ]:
#先找出来有哪些prj是有符合要求的突变，可以用的
import os
import csv


all_prj_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/fastq_result')
csv_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/09snp_isnv_all_mut')
not_empty_prj_list = []
for i in csv_list:
    csv_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/09snp_isnv_all_mut/{i}'
    total = len(open(csv_path).readlines())
    if total != 1:
        not_empty_prj_list.append(i.split('.')[0])
print(len(not_empty_prj_list))
print(not_empty_prj_list)

with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/11consensus/00script/01write_run_spades.sh','w')as f:
    for i in all_prj_list:
        if i in not_empty_prj_list:
            prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/fastq_result/{i}'
            fq_name_list = []
            all_fq_list = os.listdir(prj_path)
            for n in all_fq_list:
                n1 = n.split('_')[0]
                if n1 not in fq_name_list:
                    fq_name_list.append(n1)
            #新建目录建立prj和其中每个样本output的目录。
            for s in fq_name_list:           

                dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/11consensus/01spades_consensus_output/{i}/{s}'
                if not os.path.exists(dirs):
                    os.makedirs(dirs)
            for s in fq_name_list: 
                f.write(f'docker run -id -v /mnt/alamo01/users/yangjingqi/RSV_sra/group/fastq_result/{i}:/input -v /mnt/alamo01/users/yangjingqi/RSV_sra/group/11consensus/01spades_consensus_output/{i}/{s}:/output staphb/spades spades.py -1 /input/{s}_1.fastq.gz -2 /input/{s}_2.fastq.gz -o /output\n')

In [ ]:
#一致性序列

In [ ]:
#构建一致性序列先用bcftools对vcf文件排序压缩。
import os
import csv


all_prj_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/vcf_pair')
csv_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/09snp_isnv_all_mut')
not_empty_prj_list = []
for i in csv_list:
    csv_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/09snp_isnv_all_mut/{i}'
    total = len(open(csv_path).readlines())
    if total != 1:
        not_empty_prj_list.append(i.split('.')[0])
print(len(not_empty_prj_list))
print(not_empty_prj_list)

with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/00script/01write_run_bcftools_sort_gz.sh','w')as f:
    for i in all_prj_list:
        if i in not_empty_prj_list:
            dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz/{i}'
            if not os.path.exists(dirs):
                os.makedirs(dirs)
            prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/vcf_pair/{i}'
            vcf_name_list = []
            all_vcf_list = os.listdir(prj_path)
            for n in all_vcf_list:
                n1 = n.split('.')[0]
                f.write(f'bcftools sort /mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/vcf_pair/{i}/{n1}.vcf -Oz -o /mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz/{i}/{n1}_sorted.vcf.gz\n')
                
                
                
                
    

In [ ]:
#tabix进行索引
import os
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/00script/02tabix.sh','w')as f:
    prj_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz')
    for i in prj_list:
        prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz/{i}'
        gz_list = os.listdir(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz/{i}')
        for n in gz_list:
            n1 = n.split('_')[0]
            gz_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz/{i}/{n}'
            f.write(f'tabix {gz_path}\n')

        
    

In [ ]:
#构建一致性序列
import os
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/00script/03consensus.sh','w')as f:
    prj_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz')
    for i in prj_list:
        dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/02consensus/{i}'
        if not os.path.exists(dirs):
            os.makedirs(dirs)
        prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz/{i}'
        gz_list = os.listdir(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz/{i}')
        for n in gz_list:
            n1 = n.split('_')[0]
            gz_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz/{i}/{n1}_sorted.vcf.gz'
            f.write(f'bcftools consensus -f /mnt/alamo01/users/yangjingqi/RSV_sra/index/RSV.fasta {gz_path} > /mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/02consensus/{i}/{n1}.fa\n')

In [ ]:
#现在的fa文件都是参考基因组的名字，先改一下。
import os
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/00script/03consensus.sh','w')as f:
    prj_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/01write_run_bcftools_sort_gz')
    for i in prj_list:
        dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/03consensus_right_name/{i}'
        if not os.path.exists(dirs):
            os.makedirs(dirs)
            
prj_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/02consensus')
for i in prj_list:
    prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/02consensus/{i}'
    fa_list = os.listdir(f'{prj_path}')
    for n in fa_list:
        n1 = n.split('.')[0]
        fa_path = f'{prj_path}/{n}'  
        f = open(fa_path,"r")
        lines = f.readlines() #读取全部内容 ，并以列表方式返回
        cov_seq = {}
        seq_id = ''
        for line in lines:
            line = line.strip()
            if line.startswith('>'):
                seq_id = line
                cov_seq[seq_id] = ''
                seq = ''
            else:
                seq += line
                cov_seq[seq_id] = seq
        with open(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/03consensus_right_name/{i}/{n1}.fasta','w')as f:        
            for k,v in cov_seq.items():
                f.write(f'>{n1}')
                f.write('\n')
                f.write(v)
                f.write('\n')

In [ ]:
#把各个prj中的一致性序列拼在一个文件中，记得在第一行加上参考基因组,每个fasta的名字要改一下。
import os

prj_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/03consensus_right_name')
fa_path_list = []
# ref_path = '/mnt/alamo01/users/yangjingqi/RSV_sra/index/RSV.fasta'
# fa_path_list.append(ref_path)
for i in prj_list:
    prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/03consensus_right_name/{i}'
    fa_list = os.listdir(f'{prj_path}')
    for n in fa_list:
        fa_path = f'{prj_path}/{n}'
        fa_path_list.append(fa_path)
# print(fa_path_list)
        
for i in fa_path_list:
    os.system(f'cat {i} >> /mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/04consensus_all_in_one/RSV.fasta')
    
    
    
    
# list_str = ' '.join(fa_path_list)
# print(list_str)
# command = f'cat {list_str} >/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/04consensus_all_in_one/RSV_consensus.fa'
# print(command)

# os.system(f'/usr/bin/cat {list_str} >/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/04consensus_all_in_one/RSV_consensus.fa')
# os.system(f'cat /mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/03consensus_right_name/PRJNA61367/SRR527716.fasta /mnt/alamo01/users/yangjingqi/RSV_sra/index/RSV.fasta >/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/04consensus_all_in_one/hh.fasta')





In [46]:
#找一下这些cluster都包含几个序列
#chatgpt写的
clstr_file = "/mnt/alamo01/users/yangjingqi/RSV_sra/group/08workflow_result_pair_each_project/12consensus_bcf/04consensus_all_in_one/cdhit/RSV_all_with_ref_986.fasta.clstr"
clusters = []
current_cluster = None

with open(clstr_file, "r") as file:
    for line in file:
        line = line.strip()
        if line.startswith(">"):
            if current_cluster is not None:
                clusters.append(current_cluster)

            current_cluster = []
        else:
            sequence_id = line.split(">")[1].split("...")[0]
            status = line.split(">")[1].split("...")[1].strip()  #是不是在去冗余中留下，*是留下了
            current_cluster.append(sequence_id)
            
if current_cluster is not None:
    clusters.append(current_cluster)    
# print(clusters)
#--------------------------------------------------------------------------------------------
dict_cluster_num = {}
for i, cluster in enumerate(clusters):    
    dict_cluster_num[f"Cluster {i}"] = len(cluster)
# print(dict_cluster_num)

dict_cluster_num_sorted = dict(sorted(dict_cluster_num.items(),key = lambda x:x[1]))
print(dict_cluster_num_sorted)
#--------------------------------------------------------------------------------------------

dict_cluster_id = {}
for i, cluster in enumerate(clusters):  
    dict_cluster_id[f"Cluster {i}"] = cluster
# print(dict_cluster_id)
print(dict_cluster_id['Cluster 100'])
print(dict_cluster_id['Cluster 108'])
#--------------------------------------------------------------------------------------------
# 看看带星号的序列都有哪些
list_seq_in_tree = []
with open(clstr_file, "r") as file:
    for line in file:
        line = line.strip()
        if line.startswith(">"):
            pass
        else:
            sequence_id = line.split(">")[1].split("...")[0]
            status = line.split(">")[1].split("...")[1].strip()    #是不是在去冗余中留下，*是留下了
            if status == '*':
                list_seq_in_tree.append(sequence_id)

# print(len(list_seq_in_tree))

#-------------
#每个cluster都是什么代表的seq
dict_cluster_seq_tree = {}
for k,v in dict_cluster_id.items():
    for i in v:
        if i in list_seq_in_tree:
            dict_cluster_seq_tree[k] = i
# print(dict_cluster_seq_tree)

{'Cluster 6': 1, 'Cluster 7': 1, 'Cluster 10': 1, 'Cluster 12': 1, 'Cluster 18': 1, 'Cluster 27': 1, 'Cluster 35': 1, 'Cluster 36': 1, 'Cluster 37': 1, 'Cluster 38': 1, 'Cluster 39': 1, 'Cluster 45': 1, 'Cluster 48': 1, 'Cluster 50': 1, 'Cluster 51': 1, 'Cluster 53': 1, 'Cluster 54': 1, 'Cluster 58': 1, 'Cluster 63': 1, 'Cluster 67': 1, 'Cluster 68': 1, 'Cluster 71': 1, 'Cluster 78': 1, 'Cluster 86': 1, 'Cluster 87': 1, 'Cluster 89': 1, 'Cluster 104': 1, 'Cluster 105': 1, 'Cluster 107': 1, 'Cluster 112': 1, 'Cluster 115': 1, 'Cluster 119': 1, 'Cluster 120': 1, 'Cluster 121': 1, 'Cluster 123': 1, 'Cluster 125': 1, 'Cluster 134': 1, 'Cluster 137': 1, 'Cluster 139': 1, 'Cluster 140': 1, 'Cluster 141': 1, 'Cluster 142': 1, 'Cluster 144': 1, 'Cluster 147': 1, 'Cluster 149': 1, 'Cluster 151': 1, 'Cluster 153': 1, 'Cluster 154': 1, 'Cluster 155': 1, 'Cluster 158': 1, 'Cluster 159': 1, 'Cluster 160': 1, 'Cluster 161': 1, 'Cluster 163': 1, 'Cluster 164': 1, 'Cluster 167': 1, 'Cluster 168': 1, '